# Measurement and Instrumentation Data Center

In [5]:
import scrapy
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
hostname = os.getenv('HOSTNAME')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
database = os.getenv('DATABASE')

## psycopg2

In [4]:
def queryNREL(conn):
    cur = conn.cursor()
    cur.execute("""SELECT * FROM stations""")
    rows = cur.fetchall()
    
    for row in rows:
        print(row)

In [5]:
conn = psycopg2.connect(host = hostname,
                        user = username,
                        password = password,
                        database = database)

In [6]:
queryNREL(conn)
conn.close()

(1, 'NREL Solar Radiation Research Laboratory (BMS)', '(39.742,-105.18)')


## sqlalchemy

In [7]:
TESTDATA = StringIO('DATE (MM/DD/YYYY),MST,Global PSP (vent/cor) [W/m^2],Global LI-200 [W/m^2],Global 40-South CMP22 [W/m^2],Global 40-South LI-200R [W/m^2],Global 90-North PSP [W/m^2],Global 90-North LI-200 [W/m^2],Global 90-East PSP [W/m^2],Global 90-East LI-200 [W/m^2],Global 90-South PSP [W/m^2],Global 90-South LI-200 [W/m^2],Global 90-West PSP [W/m^2],Global 90-West LI-200 [W/m^2],Global Normal CMP22 [W/m^2],Global Normal LI-200R [W/m^2],Direct sNIP [W/m^2],Direct NIP [W/m^2],Diffuse CM22-2 (vent/cor) [W/m^2],Diffuse CM22-1 (vent/cor) [W/m^2]\n12/29/2020,07:24,9.60577,2.89732,5.72671,7.09437,1.50504,5.16036,5.86544,9.50543,4.43471,7.68627,1.49090,4.56097,8.41596,9.76368,0.553023,-0.210355,5.04128,5.00532\n12/29/2020,07:25,10.1379,3.17354,6.42456,7.74575,2.25976,5.57925,6.86880,10.4399,5.31332,8.43088,2.18842,4.97855,9.63116,10.8061,0.592524,0.704445,5.39809,5.53059\n12/29/2020,07:26,10.8743,3.52482,7.25354,8.44308,3.54853,6.00175,8.57102,11.5875,6.71704,9.30772,3.39178,5.37993,11.3402,12.0880,1.57513,2.48023,5.67426,5.98470\n12/29/2020,07:27,11.7694,3.83106,8.09946,9.14878,4.97414,6.40983,10.5878,12.7679,8.55749,10.2055,4.73893,5.78458,13.2295,13.4372,2.14296,3.94783,5.93630,6.36775\n12/29/2020,07:28,12.571,4.03222,8.91154,9.84614,6.28499,6.78902,12.4647,13.8369,10.2212,11.0545,5.86776,6.18599,14.9384,14.6603,2.51329,4.91644,6.22500,6.73611\n12/29/2020,07:29,13.3013,4.24538,9.85896,10.6771,7.43696,7.15738,14.4865,15.1517,11.9369,12.1401,6.93803,6.63919,16.8750,16.2659,3.16999,5.81165,6.51389,7.06947\n12/29/2020,07:30,13.8146,4.64770,11.6269,12.2138,8.42121,7.66657,18.0457,18.4405,14.5052,14.548,7.98700,7.23158,20.5849,20.3723,5.66352,8.30654,6.86454,7.43788\n12/29/2020,07:31,14.6877,5.01099,13.1073,13.4498,9.15389,8.1252,20.6665,20.7521,16.4236,16.3156,8.82831,7.77865,23.3588,23.2011,7.23865,9.85730,7.23046,7.82232\n12/29/2020,07:32,15.4574,5.11609,13.7544,13.9676,9.83359,8.54411,21.5501,20.9489,17.1723,16.681,9.56312,8.25450,24.6771,24.2434,7.06092,9.19693,7.68661,8.24682\n12/29/2020,07:33,15.9042,4.77382,11.6184,12.4726,10.0587,8.61631,17.2570,15.8633,14.5936,13.4415,9.83998,8.44870,18.3607,16.6905,0.987541,2.57318,8.24020,8.73741\n12/29/2020,07:34,16.3688,6.16994,15.2009,15.7548,10.8090,9.68166,24.0162,23.5327,18.6333,18.6818,10.7772,9.61733,27.5926,27.5722,12.6208,13.9666,8.96180,9.36759\n12/29/2020,07:35,17.2282,18.5189,75.2687,63.6664,18.4844,21.3495,142.300,158.975,90.6241,108.705,18.1838,19.0566,164.375,193.426,144.695,145.938,10.6334,10.5614\n12/29/2020,07:36,17.0045,22.8484,107.451,87.1128,21.7196,26.5677,198.260,221.953,126.783,151.525,21.3840,23.5594,240.523,280.783,197.494,196.399,11.7131,11.3433\n12/29/2020,07:37,16.3081,14.2314,51.8795,44.8593,13.7706,16.6622,87.5137,95.7967,58.6243,68.2415,13.9987,15.6124,121.986,138.761,95.5300,93.7453,11.8318,11.4412\n12/29/2020,07:38,16.6795,13.3097,46.5843,40.8132,13.4926,16.2181,77.4252,83.9306,51.8969,60.4926,14.1531,15.292,90.0908,100.395,75.4927,74.5784,12.3553,12.0799\n12/29/2020,07:39,17.5417,11.5893,43.1076,38.4581,13.7751,16.1097,71.9340,76.7464,49.0166,56.6094,15.3192,15.9297,81.8378,91.8589,55.5840,55.6319,13.0302,12.8651\n12/29/2020,07:40,18.7571,14.8860,57.116,49.7869,16.3218,19.4249,96.8941,104.178,65.3418,76.0675,18.4129,19.2672,114.712,129.140,85.2400,85.8156,14.0532,13.9048\n12/29/2020,07:41,20.5243,23.5960,97.0089,83.0878,22.5451,28.6298,170.455,190.430,112.591,135.704,24.3606,26.7480,198.563,229.640,164.476,165.301,16.2231,15.7783\n12/29/2020,07:42,21.6742,39.5117,178.808,149.377,33.7955,44.9779,318.147,365.413,208.373,256.592,34.8610,40.0007,370.933,433.137,319.046,319.897,18.4485,17.7079\n12/29/2020,07:43,20.7206,45.8768,215.420,179.455,38.3726,51.6516,382.983,444.059,251.26,311.117,39.1900,45.6818,448.009,525.837,388.766,388.512,17.8955,17.0894\n12/29/2020,07:44,20.8683,48.5790,228.569,191.067,40.3411,54.6669,404.623,470.237,266.410,330.144,41.1229,48.3071,476.109,559.521,411.855,410.458,18.1932,17.3995\n12/29/2020,07:45,21.1148,50.7469,237.599,199.464,41.8814,56.9672,417.193,485.763,276.470,343.334,42.7469,50.4015,490.546,576.489,427.385,425.883,18.6205,17.8467\n12/29/2020,07:46,21.5719,52.4612,244.139,205.708,43.2321,58.6177,424.602,494.529,283.578,351.700,44.2859,52.0525,499.391,586.488,436.821,435.095,18.9824,18.3621\n12/29/2020,07:47,22.8137,53.8423,250.377,211.808,45.0108,60.3655,431.885,502.523,290.539,359.923,46.1548,53.7002,509.610,597.638,442.786,441.327,19.6230,19.0282\n12/29/2020,07:48,23.6656,55.3254,258.828,219.470,47.2485,62.7020,442.647,514.713,299.824,370.834,48.3592,55.7589,523.414,612.482,451.796,449.907,20.3877,19.7438\n12/29/2020,07:49,24.1821,57.2559,267.130,227.145,49.0845,64.7747,453.075,526.939,308.865,381.356,50.2494,57.6849,536.066,627.208,461.894,458.879,20.8659,20.2274\n12/29/2020,07:50,24.4272,59.0964,276.273,235.788,50.9866,67.0965,464.664,540.384,318.908,393.898,52.1608,59.7759,550.171,642.873,477.487,475.213,21.3613,20.6150\n12/29/2020,07:51,24.9110,61.4775,286.631,245.404,52.9640,69.6532,478.432,556.654,330.279,408.154,54.0564,61.9447,566.544,661.101,491.936,489.783,21.7804,21.0685\n12/29/2020,07:52,25.6473,63.4801,295.856,253.965,55.0340,71.9572,490.473,570.528,340.303,420.277,56.1011,64.0035,580.418,676.371,504.394,502.223,22.2062,21.5756\n12/29/2020,07:53,26.5981,65.0833,303.079,260.770,57.2362,73.8709,498.543,578.507,348.157,428.898,58.2734,65.7934,590.620,686.879,512.812,510.827,22.6134,22.1050\n12/29/2020,07:54,27.2247,66.2572,309.750,267.084,59.4299,75.6550,505.333,585.367,355.358,436.707,60.3288,67.4217,598.857,695.132,520.426,518.923,22.9533,22.5138\n12/29/2020,07:55,28.1107,67.8124,317.685,274.843,61.6633,77.7387,514.309,595.149,364.036,446.729,62.4961,69.3220,610.802,707.987,529.392,528.105,23.3972,23.0314\n12/29/2020,07:56,28.6679,69.4788,326.151,282.801,63.8390,79.8655,523.837,605.098,373.102,457.281,64.5246,71.2155,621.939,719.852,540.680,539.875,23.6986,23.4110\n12/29/2020,07:57,29.4560,71.0132,333.709,289.979,65.9003,81.9059,532.099,613.660,381.098,466.286,66.3776,72.9473,632.217,730.265,548.888,548.795,24.0332,23.9156\n12/29/2020,07:58,30.3345,72.2591,339.998,296.125,68.0054,83.6354,537.948,618.862,387.840,473.331,68.2996,74.4556,639.188,736.853,555.223,555.966,24.3408,24.3060\n12/29/2020,07:59,31.2005,73.7543,347.459,303.157,70.2343,85.5964,545.566,626.630,396.076,482.180,70.4082,76.2069,648.918,746.562,563.271,564.312,24.6893,24.6702\n12/29/2020,08:00,31.9362,75.2554,356.221,311.411,72.2908,87.7700,554.071,635.753,405.183,492.370,72.4686,78.0647,661.653,759.616,573.304,574.521,25.1117,25.0712\n12/29/2020,08:01,32.6466,75.6547,359.629,314.867,73.4248,89.0986,552.571,633.963,407.760,494.355,73.5013,79.0452,663.073,758.963,570.622,571.937,25.8643,25.7322\n12/29/2020,08:02,33.6128,76.0241,364.471,320.024,74.4399,90.7632,553.044,635.465,411.253,498.258,74.3106,80.3885,663.917,757.991,564.964,565.838,27.4609,26.9459\n12/29/2020,08:03,33.9068,77.1051,371.489,327.102,75.6406,92.9518,559.400,643.906,418.335,507.671,75.3757,82.3244,672.745,767.358,567.449,568.813,28.5433,27.7961\n12/29/2020,08:04,33.7642,79.2548,380.560,335.766,76.7791,94.9775,568.824,654.981,427.267,518.889,76.4671,84.1597,686.389,782.458,582.098,583.841,28.9398,28.1071\n12/29/2020,08:05,34.0421,81.2514,389.235,343.766,77.9442,96.9636,577.454,665.377,435.808,529.682,77.6225,85.8785,699.571,796.462,595.925,597.779,29.2631,28.3326\n12/29/2020,08:06,34.4469,83.2482,398.864,352.964,79.4314,99.2780,588.350,677.497,446.012,542.384,79.1983,87.9531,712.222,809.767,608.552,610.269,29.6329,28.6476\n12/29/2020,08:07,35.0639,85.0708,408.205,361.326,81.0772,101.552,597.801,688.238,456.023,553.766,80.8485,89.9596,724.807,822.429,615.540,617.555,30.1861,29.1712\n12/29/2020,08:08,35.9294,86.5481,416.471,369.090,82.6965,103.675,604.776,695.221,464.447,563.588,82.3070,91.7103,734.657,833.105,622.582,624.590,30.8148,29.7723\n12/29/2020,08:09,36.4116,87.8993,422.666,375.039,84.1307,105.351,608.733,698.908,470.914,570.609,83.5421,93.1440,742.602,840.969,626.557,628.431,31.1636,30.1385\n12/29/2020,08:10,37.3078,89.2449,430.434,382.584,85.6620,107.286,614.837,705.901,478.893,579.908,85.1659,95.0116,751.157,849.003,630.412,631.732,32.0172,31.0107\n12/29/2020,08:11,37.8002,90.9501,437.479,389.188,87.1358,109.170,619.262,709.790,485.676,586.666,86.5712,96.5066,759.240,856.494,636.855,638.540,32.2470,31.3015\n12/29/2020,08:12,38.6698,92.3671,441.335,392.456,88.2787,110.297,618.732,707.149,489.542,589.497,87.4868,97.2995,759.893,854.603,638.538,639.503,32.7816,31.9196\n12/29/2020,08:13,41.1664,93.2612,446.439,397.387,90.2293,112.817,619.459,706.669,494.460,593.331,89.1373,98.9470,750.873,840.771,626.515,627.459,35.5331,34.5595\n12/29/2020,08:14,40.2783,94.1500,453.074,403.904,91.5840,113.991,620.800,706.365,500.717,598.979,90.6811,100.261,761.994,854.204,638.514,639.333,35.0284,34.0625\n12/29/2020,08:15,40.0631,96.9965,463.288,413.908,94.3248,116.970,631.877,719.382,512.269,612.901,93.4818,102.769,783.148,877.572,664.567,665.193,33.7422,32.9483\n12/29/2020,08:16,40.8433,98.8131,471.464,421.881,96.9104,119.879,639.655,727.538,520.683,622.364,95.6216,104.552,792.742,887.298,673.925,674.142,34.0984,33.3514\n12/29/2020,08:17,43.3522,100.674,476.056,426.986,99.5935,123.201,640.592,728.375,524.970,626.172,97.4317,106.187,796.813,890.909,673.998,674.082,35.3702,34.7094\n12/29/2020,08:18,101.458,79.0028,345.128,317.376,108.049,139.017,419.754,479.545,355.199,414.893,106.243,119.451,528.882,576.255,139.738,142.529,105.751,103.729\n12/29/2020,08:19,107.269,61.0933,107.348,110.979,72.2476,94.0742,104.785,119.811,103.221,112.554,71.3150,80.9838,139.708,148.433,9.92979,9.39266,107.287,106.537\n12/29/2020,08:20,116.578,67.8306,106.806,129.108,87.1205,121.391,103.537,128.821,99.9297,118.573,81.7027,99.0841,104.950,119.853,4.39952,3.95274,119.531,118.551\n12/29/2020,08:21,114.763,68.1189,101.266,128.716,90.6600,129.176,104.071,132.152,96.4257,117.533,82.9858,102.328,97.5172,113.704,3.36753,3.28743,117.569,116.548\n12/29/2020,08:22,118.298,71.7820,105.82,132.798,99.1903,141.940,122.179,154.597,102.107,125.517,86.1372,106.292,102.193,118.814,2.48368,2.56342,117.315,116.280\n12/29/2020,08:23,242.872,151.721,237.791,289.506,176.392,255.720,242.594,307.728,225.386,289.429,170.021,216.171,237.021,283.181,5.23400,6.23242,244.738,242.710\n12/29/2020,08:24,163.417,97.2124,212.005,238.006,134.398,183.637,183.164,224.144,191.205,231.435,136.445,164.009,218.833,244.427,1.92572,2.20141,179.940,177.734\n12/29/2020,08:25,114.355,65.5640,124.642,138.357,88.0494,117.077,120.615,143.696,115.715,135.278,89.5698,102.6,129.362,141.500,1.26406,1.05179,119.514,118.337\n12/29/2020,08:26,133.057,76.8893,128.292,150.921,97.6657,133.387,118.638,143.692,118.642,141.342,97.1672,114.084,130.958,146.256,0.938173,0.259278,136.993,135.725\n12/29/2020,08:27,110.562,63.5135,102.671,123.255,86.3940,117.693,105.370,127.112,97.0102,114.422,82.5520,95.6533,100.066,111.524,0.711037,-0.376687,111.123,110.073\n12/29/2020,08:28,144.646,83.9122,148.172,173.045,109.329,148.960,140.674,169.747,136.316,163.191,108.097,127.043,145.324,162.570,0.765354,-0.528340,148.289,146.835\n12/29/2020,08:29,130.725,73.6739,111.890,130.490,86.3716,114.945,98.7860,116.085,103.804,120.792,93.1894,107.435,117.703,129.469,0.454275,-1.01755,134.457,133.276\n12/29/2020,08:30,112.530,63.3906,105.533,123.730,85.5593,114.454,100.378,118.944,98.9590,115.065,87.6147,100.420,104.775,115.083,0.454275,-1.23769,115.701,114.622\n12/29/2020,08:31,96.7828,54.4614,82.4252,101.784,77.8269,104.646,88.2483,104.842,79.4382,91.7191,74.7245,85.2526,79.8810,88.1815,0.330831,-1.66330,98.5603,97.6356\n12/29/2020,08:32,68.0611,36.3505,59.4271,70.3443,54.2911,68.7684,62.7519,71.0235,57.0793,62.8379,54.3115,58.9991,56.6272,59.5230,0.177759,-1.97638,69.1068,68.4577\n12/29/2020,08:33,96.3137,53.3774,71.9322,89.3872,68.0953,90.5559,72.8693,84.6817,69.2175,79.5239,69.4531,79.1993,71.0791,78.0695,0.113569,-2.07423,93.6676,93.0203\n12/29/2020,08:34,139.914,79.7358,107.647,134.012,97.2441,133.376,103.107,123.355,101.567,120.328,100.945,119.215,104.667,118.004,0.138258,-2.00574,138.132,137.232\n12/29/2020,08:35,127.194,72.0344,105.245,132.404,94.3933,128.513,101.335,121.221,99.7582,117.76,96.7710,113.483,99.8505,112.726,0.187635,-1.97638,129.276,128.273\n12/29/2020,08:36,87.1559,47.3782,67.6398,83.1537,63.4263,82.2439,65.7616,74.6462,65.2251,73.1606,65.7743,73.9916,64.6435,69.9881,0.103693,-2.23076,86.3429,85.7501\n12/29/2020,08:37,97.6061,53.1817,75.0578,92.5899,70.3066,92.0077,73.5883,84.2066,71.9365,81.229,70.4596,79.1478,72.3847,78.7675,0.009875,-2.31392,95.1795,94.5579\n12/29/2020,08:38,105.470,57.9015,77.798,98.3848,77.5617,102.656,80.3264,93.1076,75.3413,85.5430,76.6088,86.8639,74.7115,82.1630,0.004938,-2.30902,103.444,102.872\n12/29/2020,08:39,79.6728,43.3248,63.2033,77.7256,62.7067,81.1314,68.3070,77.8490,61.8251,68.7730,60.9983,67.2265,60.6674,65.4111,-0.004938,-2.25521,77.3408,76.9273\n12/29/2020,08:40,78.5248,42.2078,58.4201,72.4898,58.6862,75.2456,61.0594,68.5870,57.4375,63.6341,59.0602,65.4106,56.0992,60.3426,0,-2.15247,75.7815,75.4893\n12/29/2020,08:41,90.4026,48.4408,65.2880,81.6918,64.6968,83.4204,66.4849,74.8161,63.4884,70.5506,64.5492,71.8968,62.9347,67.9288,0,-2.08399,86.7377,86.4114\n12/29/2020,08:42,101.381,54.3225,67.4069,84.0802,71.3079,92.3971,71.2718,80.2718,66.3269,73.2890,70.5124,78.6877,65.7,70.7323,0.083941,-1.92255,95.2113,95.0476\n12/29/2020,08:43,146.611,81.6984,101.341,129.248,101.600,137.634,106.736,126.251,98.8279,115.293,102.862,119.846,100.126,113.18,0.281449,-1.49695,138.805,138.505\n12/29/2020,08:44,149.084,83.7610,128.429,158.558,110.621,149.553,125.054,149.096,122.351,144.324,113.138,132.120,123.801,140.065,0.493771,-1.08113,146.547,145.873\n12/29/2020,08:45,121.822,66.9866,107.330,131.678,91.9135,121.270,103.791,121.084,102.779,118.696,94.2052,107.728,101.934,113.605,0.592524,-0.963720,120.312,119.725\n12/29/2020,08:46,90.4711,47.5730,76.2379,91.0704,69.6492,88.0108,78.0858,87.4920,73.8342,81.0170,68.2450,74.2703,72.5401,77.5160,0.681402,-0.851203,87.0923,86.8195\n12/29/2020,08:47,92.0048,47.3266,61.7450,74.9876,65.9115,82.6526,67.6040,74.0829,61.3684,65.2460,63.9273,68.7583,60.3464,63.0796,0.740652,-0.797389,85.2270,85.2254\n12/29/2020,08:48,107.766,56.8204,69.4424,86.6579,76.7375,98.8767,76.2995,85.1945,69.3901,75.4544,74.3897,82.4502,67.9546,72.4980,0.696216,-0.689769,99.6402,99.6452\n12/29/2020,08:49,115.933,61.6964,79.8887,100.654,82.5677,107.189,84.1963,95.6110,78.7529,87.8269,81.9877,91.9797,77.4391,84.5305,0.706092,-0.479414,109.310,109.168\n12/29/2020,08:50,104.795,55.1031,76.6958,92.3786,77.0910,98.6173,79.0902,88.5949,75.0413,82.2672,75.7478,83.2826,74.1546,79.1006,0.765344,-0.332655,97.5408,97.4752\n12/29/2020,08:51,134.806,71.8417,87.3664,111.156,93.3014,122.277,92.8372,105.680,86.5774,96.9641,91.6518,104.054,86.1682,94.2392,0.799911,-0.107624,125.977,125.867\n12/29/2020,08:52,114.488,60.2374,87.7431,106.776,83.9941,107.523,87.1972,97.9956,85.0597,94.0350,84.1342,93.4340,83.6995,89.8096,0.888790,0.068488,109.095,108.806\n12/29/2020,08:53,98.8711,50.8338,71.6764,86.7803,72.5592,90.9160,74.5237,81.9138,70.7115,76.3497,72.1387,78.6450,68.7922,72.5950,0.873976,0.156544,92.7906,92.6562\n12/29/2020,08:54,81.3680,39.6318,60.2449,69.8785,61.4465,73.1575,63.7020,67.3139,59.3473,61.2356,60.3667,62.7101,57.8521,58.4526,0.864100,0.166328,76.0299,75.9603\n12/29/2020,08:55,98.8298,48.7411,67.7053,76.5090,71.1426,85.7525,77.2491,82.9728,66.6827,69.2798,68.0177,70.6954,69.1830,70.6449,0.893726,0.293519,88.0558,88.1315\n12/29/2020,08:56,111.565,55.7877,103.100,113.946,78.6278,94.2023,96.8162,105.383,95.8524,103.544,83.5702,87.6922,103.009,106.370,1.02211,0.596822,108.388,108.003\n12/29/2020,08:57,89.5481,42.9972,73.6099,80.0504,67.5773,78.9574,72.1384,75.0097,71.4193,73.9950,68.0769,69.9127,71.2616,70.9481,0.982600,0.557684,82.1097,82.1380\n12/29/2020,08:58,166.610,87.2407,109.453,128.648,109.310,139.803,121.670,138.761,104.514,116.283,104.192,114.180,111.635,120.391,1.09123,0.787608,147.696,147.855\n12/29/2020,08:59,165.213,87.6912,179.214,197.066,118.203,148.447,165.262,188.912,161.320,184.245,125.878,138.580,173.828,187.699,1.84670,1.72198,165.770,164.698\n12/29/2020,09:00,131.937,67.6291,127.250,141.604,93.2898,114.140,117.602,129.604,119.606,132.855,101.674,110.092,124.422,131.873,1.71832,1.23767,127.105,126.585\n12/29/2020,09:01,143.192,73.7062,126.946,142.795,102.289,127.421,119.375,133.814,120.033,133.972,105.838,115.071,126.329,134.475,1.87633,1.17408,135.167,134.775\n12/29/2020,09:02,154.641,79.0746,148.735,160.841,110.225,135.51,147.029,166.244,136.721,152.762,110.976,118.395,150.494,160.630,2.12322,1.41868,146.937,146.497\n12/29/2020,09:03,153.688,77.0842,173.773,175.869,100.762,117.903,151.955,165.906,156.523,173.864,113.292,119.434,175.358,183.409,4.45384,4.42238,148.366,147.814\n12/29/2020,09:04,155.128,78.8826,182.519,182.182,107.307,127.349,159.737,174.077,167.274,186.72,113.415,118.123,192.275,199.703,3.50579,3.23362,146.619,146.213\n12/29/2020,09:05,184.56,96.0834,202.164,208.75,119.223,144.371,186.298,208.228,182.532,205.808,124.819,130.964,219.248,231.141,3.12065,2.84225,178.141,177.575\n12/29/2020,09:06,154.191,80.2755,203.564,202.729,113.627,135.821,178.935,197.792,186.233,207.857,112.866,115.612,229.050,237.718,4.22175,4.46150,152.939,152.245\n12/29/2020,09:07,115.359,57.4752,144.942,142.921,94.3364,108.883,136.257,148.214,134.990,145.942,90.9724,89.0369,154.194,156.674,2.77994,2.79822,111.075,111.064\n12/29/2020,09:08,112.117,53.9053,129.103,126.407,86.9839,98.4760,111.339,118.673,121.889,130.174,86.4256,84.5023,138.427,138.492,2.33060,2.24054,105.613,105.642\n12/29/2020,09:09,109.822,51.7347,134.436,126.649,86.6484,97.1362,109.557,115.690,124.697,134.252,82.9008,79.4494,141.576,139.430,2.33061,2.24543,102.726,102.582\n12/29/2020,09:10,119.545,57.1780,126.439,124.774,83.4531,93.1424,122.600,131.178,114.637,122.592,79.5784,75.9632,138.338,138.632,2.28123,2.26500,111.733,111.868\n12/29/2020,09:11,116.029,55.0764,152.779,141.584,84.0532,91.7232,138.039,145.316,135.567,145.125,82.0435,78.8149,169.516,169.579,4.20201,4.53489,112.376,112.030\n12/29/2020,09:12,141.685,71.1695,173.608,166.295,87.7783,97.0532,156.363,165.713,157.22,172.539,91.4306,90.4355,198.726,204.737,31.7546,34.5376,125.454,124.744\n12/29/2020,09:13,162.048,83.2601,249.512,231.528,97.5278,108.320,195.244,203.708,228.781,253.584,107.968,108.585,369.336,379.118,120.994,124.932,134.212,132.709\n12/29/2020,09:14,144.076,72.7576,187.621,183.131,91.1903,101.961,160.842,168.881,174.850,191.895,101.398,102.212,219.270,224.527,13.2627,14.5244,137.601,136.673\n12/29/2020,09:15,229.743,127.987,490.439,453.842,148.048,175.257,411.128,447.468,462.595,531.215,162.698,172.430,515.612,541.710,200.536,204.583,190.693,188.496\n12/29/2020,09:16,140.645,74.7874,285.338,260.038,104.514,115.329,245.764,257.554,269.279,295.622,116.046,116.609,303.714,307.886,28.9895,30.8784,137.601,136.698\n12/29/2020,09:17,136.407,70.7821,219.424,205.686,94.3941,104.207,201.917,210.019,209.488,229.075,108.485,109.925,227.879,233.554,20.8422,24.6166,129.607,128.830\n12/29/2020,09:18,159.822,83.5033,216.629,212.317,102.709,116.629,197.684,211.697,201.622,222.137,116.556,119.839,240.112,250.357,15.1292,17.3813,156.729,155.793\n12/29/2020,09:19,163.570,87.2655,254.840,242.221,110.318,125.912,226.576,246.740,230.293,256.503,118.505,121.073,274.723,286.577,22.2495,26.1087,160.915,159.511\n12/29/2020,09:20,133.738,71.2203,299.705,273.546,109.378,122.713,270.823,285.286,285.276,314.863,115.982,116.583,256.625,262.957,72.4712,77.783,124.568,123.472\n12/29/2020,09:21,213.664,119.130,375.164,350.541,118.288,135.810,316.386,336.955,359.373,407.888,136.720,143.194,350.562,366.074,196.758,212.670,149.454,147.894\n12/29/2020,09:22,196.457,106.265,444.651,402.303,132.773,151.255,342.697,361.332,416.460,464.733,156.064,164.311,417.589,433.050,170.100,174.390,170.449,168.367\n12/29/2020,09:23,151.583,78.5196,246.593,240.129,105.388,119.185,193.591,209.124,225.822,253.128,126.215,132.146,268.088,279.750,8.52256,9.95038,155.258,154.530\n12/29/2020,09:24,174.116,93.2136,356.339,343.872,131.007,153.305,296.099,326.115,332.864,383.642,148.769,157.575,345.652,367.720,23.2914,27.4884,175.122,173.973\n12/29/2020,09:25,168.433,91.4717,366.040,347.429,130.685,152.637,298.885,323.678,350.113,401.485,149.908,159.08,394.960,417.468,29.2067,33.5739,171.756,170.532\n12/29/2020,09:26,183.895,100.059,366.565,348.063,135.575,159.115,290.438,314.039,354.501,405.092,155.360,165.690,429.145,449.779,25.4984,29.0194,186.935,185.914\n12/29/2020,09:27,200.688,111.312,403.135,387.375,146.706,174.747,322.345,354.983,382.984,445.829,166.935,179.653,450.746,481.640,45.8171,51.8258,199.949,198.626\n12/29/2020,09:28,210.283,119.349,516.256,487.280,167.775,200.931,414.965,456.027,499.597,583.076,191.204,208.031,564.948,602.164,60.2204,66.7464,213.787,212.192\n12/29/2020,09:29,223.719,127.332,524.586,497.262,170.961,204.790,412.732,452.917,501.326,585.833,196.336,214.689,621.454,664.565,57.0405,64.5449,227.268,225.492\n12/29/2020,09:30,244.355,140.369,620.240,584.325,190.401,230.283,491.442,540.541,601.705,702.019,214.544,238.025,681.453,728.692,83.3091,91.9450,243.332,241.3\n12/29/2020,09:31,266.930,154.249,607.406,573.946,190.612,229.715,480.324,527.194,590.296,686.752,211.540,233.949,741.915,796.482,107.366,117.672,253.515,251.319\n12/29/2020,09:32,281.062,163.688,649.151,608.718,196.548,237.095,518.274,567.180,633.850,734.981,208.042,228.572,732.456,783.104,180.898,198.394,242.293,240.188\n12/29/2020,09:33,287.144,166.486,791.979,723.414,217.674,264.444,638.116,697.542,782.711,904.607,223.264,245.811,813.975,869.700,278.398,287.830,226.652,224.584\n12/29/2020,09:34,329.477,187.137,802.580,720.760,216.099,260.693,650.147,700.793,797.020,905.507,209.687,227.870,856.105,905.681,345.432,354.292,191.544,190.098\n12/29/2020,09:35,297.142,169.459,890.021,793.037,220.354,264.293,711.634,766.342,890.265,1010.34,213.186,231.078,893.372,942.198,480.908,487.628,160.999,159.756\n12/29/2020,09:36,315.406,180.423,867.205,766.429,210.733,249.937,690.496,738.549,879.510,987.466,209.544,225.405,1026.56,1075.71,616.902,623.869,140.654,139.448\n12/29/2020,09:37,285.716,163.069,904.516,797.260,215.570,255.649,711.790,761.582,915.365,1025.98,214.523,231.619,1055.42,1103.32,623.405,623.194,133.547,132.129\n12/29/2020,09:38,249.910,138.309,875.518,769.078,208.672,246.155,692.666,737.220,896.809,997.021,208.298,224.174,1032.06,1078.93,427.536,429.892,120.035,119.113\n12/29/2020,09:39,337.758,192.871,837.251,742.652,205.802,243.623,667.015,709.606,857.591,956.238,204.096,220.551,1025.67,1075.81,744.235,749.451,111.415,111.179\n12/29/2020,09:40,332.816,189.331,883.489,780.440,212.814,252.320,695.399,741.629,898.385,1003.70,209.430,226.829,1042.50,1089.89,758.766,761.167,102.950,103.017\n12/29/2020,09:41,328.704,186.626,865.414,762.294,209.896,248.164,678.472,721.366,878.183,977.717,206.677,223.508,1026.86,1070.52,756.075,758.437,101.365,101.303\n12/29/2020,09:42,337.316,191.328,857.239,753.333,208.692,246.178,669.233,710.861,868.337,965.839,205.639,222.058,1019.76,1061.35,761.828,764.459,100.832,101.066\n12/29/2020,09:43,335.178,190.292,862.637,759.828,209.697,248.084,667.465,710.729,872.374,974.462,207.417,224.304,1031.55,1074.38,777.386,780.216,97.8198,98.0119\n12/29/2020,09:44,342.654,193.469,879.916,775.155,213.390,252.621,673.170,717.406,887.455,991.887,210.434,227.526,1036.06,1077.46,764.464,766.656,96.03,96.1925\n12/29/2020,09:45,340.131,194.784,887.955,782.708,215.330,254.822,672.433,717.270,894.098,999.741,212.712,230.127,1049.71,1091.30,785.402,788.163,96.8662,96.9281\n12/29/2020,09:46,339.997,197.027,880.216,773.545,214.667,253.596,660.682,703.275,883.117,984.636,211.501,228.491,1028.11,1064.26,739.255,742.394,99.3688,99.6616\n12/29/2020,09:47,345.046,203.2,893.122,786.942,218.297,258.744,664.511,709.585,893.295,998.981,215.876,233.950,1040.98,1079.31,749.609,751.434,103.982,104.539\n12/29/2020,09:48,360.052,211.075,905.291,798.343,221.844,263.230,666.583,712.966,902.620,1010.73,219.990,238.958,1047.30,1084.83,778.060,780.776,110.475,111.034\n12/29/2020,09:49,359.726,200.224,903.644,796.430,222.765,264.109,660.769,706.395,900.171,1006.50,219.632,238.432,1040.03,1076.98,766.101,768.140,110.678,111.287\n12/29/2020,09:50,363.533,189.218,907.447,799.429,223.925,265.130,659.629,705.606,904.306,1009.57,221.351,240.124,1052.90,1089.95,786.384,788.167,107.449,108.265\n12/29/2020,09:51,371.181,179.835,914.871,807.068,226.073,267.502,661.783,708.325,912.849,1018.85,223.858,242.791,1062.49,1100.54,812.228,814.305,104.060,105.037\n12/29/2020,09:52,371.376,164.319,920.034,811.673,226.998,268.842,661.602,707.934,920.253,1026.04,224.612,243.746,1071.09,1110.35,824.908,826.828,98.8273,99.7592\n12/29/2020,09:53,372.780,196.273,928.058,818.648,228.430,270.920,663.065,709.221,928.553,1034.27,225.649,245.052,1073.50,1113.43,834.669,835.873,96.3555,97.2289\n12/29/2020,09:54,372.375,332.696,927.249,818.584,228.539,271.410,658.953,704.869,928.132,1033.48,225.956,245.555,1072.46,1113.24,833.973,834.694,95.0656,95.6666\n12/29/2020,09:55,367.941,494.922,927.957,819.923,229.328,272.657,655.430,701.403,929.170,1034.92,226.844,246.758,1077.95,1120.03,846.584,847.491,93.4202,94.4065\n12/29/2020,09:56,383.018,563.196,940.510,832.713,232.313,276.627,659.477,706.561,941.748,1050.35,229.770,250.281,1084.98,1128.43,857.990,858.772,93.6096,94.5675\n12/29/2020,09:57,384.194,573.395,944.758,837.225,233.927,278.661,657.756,705.287,943.234,1055.51,231.147,251.955,1086.08,1130.64,857.915,858.136,94.2973,95.1458\n12/29/2020,09:58,385.120,591.271,942.997,836.289,234.402,279.035,652.347,699.720,942.552,1053.42,231.369,252.381,1082.04,1126.81,856.049,856.497,94.0673,95.0167\n12/29/2020,09:59,389.7,575.407,944.750,838.592,235.698,280.694,649.038,696.712,946.049,1054.65,232.438,253.748,1082.90,1128.99,857.254,862.778,94.0126,94.9125\n12/29/2020,10:00,393.711,515.047,950.765,845.375,237.920,283.411,646.993,696.403,953.123,1061.97,234.768,256.378,1085.02,1131.98,864.087,865.068,96.2235,96.7686\n12/29/2020,10:01,392.181,417.543,954.200,849.338,239.635,285.221,640.130,693.739,956.587,1063.91,236.598,258.392,1082.00,1129.43,858.152,859.603,99.3983,100.166\n12/29/2020,10:02,396.105,384.256,949.830,846.465,239.699,285.272,629.323,686.46,952.580,1057.47,237.191,258.972,1073.61,1121.45,841.650,843.357,103.801,104.425\n12/29/2020,10:03,395.071,380.062,942.065,840.286,238.787,284.262,616.016,676.530,944.144,1046.61,236.577,258.235,1070.38,1118.79,828.220,830.281,107.460,107.986\n12/29/2020,10:04,398.202,386.201,944.539,843.268,239.082,284.795,610.993,674.449,947.285,1049.21,236.964,258.586,1063.94,1112.29,832.469,835.151,106.731,106.972\n12/29/2020,10:05,392.181,379.703,936.428,836.163,237.610,283.099,601.375,664.703,941.002,1041.69,235.285,256.551,1052.93,1100.90,818.810,821.697,105.172,105.171\n12/29/2020,10:06,391.656,379.193,934.475,834.682,237.449,282.840,595.021,659.161,939.970,1039.61,235.262,256.099,1053.01,1102.26,818.258,820.934,105.001,104.980\n12/29/2020,10:07,395.384,382.339,931.770,832.877,237.143,282.456,589.887,653.796,937.422,1035.98,234.813,255.204,1041.87,1090.00,809.227,812.726,104.851,104.892\n12/29/2020,10:08,402.537,391.253,939.397,840.457,238.907,284.281,591.071,655.520,952.022,1043.98,236.430,256.565,1047.18,1097.74,825.467,740.139,107.589,105.167\n12/29/2020,10:09,396.922,389.987,940.859,842.644,239.197,284.524,587.914,652.920,956.361,1046.74,237.786,257.858,1046.85,1099.55,819.221,727.244,110.113,107.321\n12/29/2020,10:10,398.061,392.497,943.900,846.982,240.254,285.389,584.929,652.705,958.251,1048.21,239.248,259.254,1047.91,1102.13,818.466,736.158,112.858,109.993\n12/29/2020,10:11,404.368,399.111,950.686,854.355,242.299,287.577,554.437,655.016,962.192,1053.33,240.640,260.763,1051.34,1107.14,820.595,742.967,115.454,112.421\n12/29/2020,10:12,413.151,408.863,966.188,871.693,245.694,291.712,602.575,666.523,977.579,1074.13,243.135,263.829,1060.79,1121.03,841.782,767.628,116.200,112.997\n12/29/2020,10:13,415.625,412.279,987.795,893.681,249.116,295.504,608.599,679.203,999.572,1100.32,246.912,268.409,1092.53,1160.60,856.163,786.225,115.572,112.526\n12/29/2020,10:14,418.017,414.548,989.398,896.712,249.220,295.065,607.696,678.509,1000.68,1103.63,246.724,268.358,1090.51,1161.24,859.531,794.219,114.949,112.054\n12/29/2020,10:15,422.337,418.573,995.900,904.039,251.129,296.966,610.163,682.361,1005.96,1110.17,247.894,269.619,1095.43,1168.20,868.373,810.355,114.920,112.177\n12/29/2020,10:16,424.390,420.235,1001.55,909.91,252.896,299.046,612.232,685.861,1010.01,1116.50,248.839,270.668,1102.36,1176.43,867.585,818.009,116.406,113.680\n12/29/2020,10:17,427.099,423.120,1004.94,915.621,253.503,299.646,611.384,686.746,1012.32,1121.38,249.795,271.526,1106.01,1181.96,863.071,819.080,117.064,117.115\n12/29/2020,10:18,431.315,427.375,1014.05,924.756,254.066,299.341,611.155,687.463,1037.75,1131.06,251.406,272.909,1111.35,1188.89,856.628,824.950,122.847,122.192\n12/29/2020,10:19,431.300,427.740,1022.45,931.124,252.727,296.952,608.062,684.665,1029.73,1138.42,251.574,273.030,1122.04,1201.56,849.230,829.259,125.747,125.168\n12/29/2020,10:20,428.186,424.957,1021.01,928.833,249.786,293.369,599.124,675.144,1016.73,1138.59,249.842,271.025,1124.37,1205.09,830.537,818.688,129.588,128.936\n12/29/2020,10:21,434.521,432.130,1030.63,940.205,249.838,293.490,593.606,669.701,1024.93,1152.84,250.096,271.661,1110.54,1191.34,842.031,837.321,126.868,126.335\n12/29/2020,10:22,440.043,438.956,1061.73,972.007,257.427,303.344,608.698,689.391,1056.24,1195.64,256.331,279.562,1160.79,1250.85,862.378,861.534,125.343,125.416\n12/29/2020,10:23,444.551,442.949,1048.56,960.651,258.215,303.764,601.278,680.052,1042.08,1176.98,256.270,280.460,1148.12,1234.03,861.568,861.378,129.179,128.842\n12/29/2020,10:24,450.719,448.785,1052.46,968.493,261.742,307.660,602.061,682.392,1044.95,1180.87,258.764,285.832,1144.59,1229.03,854.497,855.345,136.656,136.129\n12/29/2020,10:25,459.907,458.457,1051.85,969.339,262.900,309.570,598.667,681.346,1041.06,1179.21,260.731,288.929,1144.21,1229.82,860.242,863.469,141.233,140.741\n12/29/2020,10:26,431.582,427.016,889.818,825.978,235.236,276.034,507.193,577.544,877.549,996.202,234.777,257.689,1017.39,1093.58,738.428,731.365,142.694,143.473\n12/29/2020,10:27,472.373,473.973,963.183,887.474,252.294,298.180,532.996,608.335,935.601,1063.62,247.464,271.283,1107.80,1191.44,878.486,804.461,142.420,140.519\n12/29/2020,10:28,470.906,471.360,1079.36,996.669,278.485,331.716,603.788,697.256,1064.47,1218.92,267.910,296.080,1174.83,1265.22,875.649,857.561,139.667,136.388\n12/29/2020,10:29,473.396,473.726,1080.39,999.151,280.072,333.349,605.231,700.074,1064.37,1222.19,268.659,298.595,1180.00,1271.94,879.576,883.915,143.077,142.567\n12/29/2020,10:30,447.639,440.429,943.381,875.934,255.753,302.510,527.693,607.148,921.975,1056.20,245.591,272.353,1051.13,1128.29,746.385,753.352,158.461,155.680\n12/29/2020,10:31,359.454,343.522,650.213,599.076,203.998,235.594,382.559,433.403,642.240,715.856,195.730,213.278,864.66,909.541,496.046,500.390,164.594,163.253\n12/29/2020,10:32,322.393,305.941,598.726,554.339,196.983,227.482,358.712,406.376,584.320,655.269,186.830,204.739,614.103,646.968,323.372,337.926,181.781,180.177\n12/29/2020,10:33,268.161,242.821,328.191,313.790,152.565,176.957,232.465,261.831,306.543,341.899,144.309,156.992,356.769,362.665,83.7290,97.7031,203.240,201.296\n12/29/2020,10:34,409.392,402.342,549.849,529.270,200.249,237.928,340.234,389.953,515.377,594.533,191.527,213.400,602.683,640.506,414.186,433.353,225.203,222.476\n12/29/2020,10:35,238.858,229.696,412.336,396.475,172.970,202.232,269.818,305.436,380.101,431.134,169.023,185.939,452.591,465.634,49.5500,58.0974,233.658,231.394\n12/29/2020,10:36,230.826,211.685,387.163,367.599,165.527,191.763,248.921,281.372,354.033,404.115,161.662,177.910,422.581,432.117,22.9653,32.1013,225.321,223.533\n12/29/2020,10:37,208.294,188.725,349.020,329.087,156.809,180.277,227.150,256.505,321.688,365.648,152.501,168.377,366.029,372.531,12.1863,20.6442,207.426,205.318\n12/29/2020,10:38,199.753,177.739,330.468,311.594,151.828,174.270,217.818,245.751,307.853,351.376,149.084,164.287,348.834,354.62,9.65319,18.2324,195.655,193.638\n12/29/2020,10:39,205.810,184.744,353.104,338.269,157.217,182.664,231.079,262.965,334.072,388.764,154.416,170.452,360.090,370.770,26.7130,37.0567,192.851,190.386\n12/29/2020,10:40,231.450,213.423,417.994,395.334,174.557,202.932,264.749,300.191,395.115,455.200,168.874,185.634,508.947,531.309,79.2997,94.7872,198.397,196.054\n12/29/2020,10:41,223.707,203.870,387.846,367.168,176.355,206.331,262.582,297.433,362.890,420.738,162.805,178.593,438.782,453.274,32.7272,44.6640,207.562,205.954\n12/29/2020,10:42,218.267,197.844,340.172,321.992,169.406,198.605,244.749,276.913,310.508,359.847,151.112,165.123,361.718,366.576,16.1809,25.4140,207.536,205.398\n12/29/2020,10:43,219.969,199.198,323.193,306.886,162.153,189.447,239.917,271.405,295.385,339.228,145.853,159.495,338.007,343.358,27.3352,37.2819,199.405,196.734\n12/29/2020,10:44,205.161,184.233,315.640,298.629,154.869,179.232,233.520,263.427,289.459,330.716,143.766,157.094,334.556,340.702,18.7583,26.8912,193.577,190.680\n12/29/2020,10:45,199.488,178.988,338.670,322.062,155.985,180.657,238.304,268.063,313.133,359.650,146.575,160.041,340.612,346.294,29.6362,37.8933,186.378,184.277\n12/29/2020,10:46,196.986,177.088,388.299,369.156,165.341,192.449,259.189,291.568,366.328,422.302,157.150,172.096,349.019,360.684,21.1038,30.7853,186.267,184.504\n12/29/2020,10:47,219.293,201.861,375.316,360.971,169.955,199.819,252.608,284.695,345.895,400.983,165.782,183.057,386.049,393.584,12.8380,21.1481,210.625,209.458\n12/29/2020,10:48,248.930,233.822,341.847,339.301,174.136,207.491,249.808,282.937,305.841,352.074,168.730,188.242,351.330,359.020,4.72046,9.73997,244.674,243.262\n12/29/2020,10:49,253.536,239.458,327.106,329.263,176.006,212.475,247.542,281.425,282.499,329.732,170.884,190.519,335.431,343.679,4.04894,8.60504,250.810,249.257\n12/29/2020,10:50,246.309,232.275,310.391,315.319,174.908,212.633,236.970,270.299,253.181,308.785,167.244,185.521,322.151,331.730,4.00941,8.21853,243.025,241.484\n12/29/2020,10:51,277.210,266.676,350.900,356.273,185.750,228.319,251.992,289.051,261.188,355.073,173.423,193.130,370.64,388.470,33.4826,42.9027,247.278,245.530\n12/29/2020,10:52,326.181,323.100,539.117,536.738,227.043,281.981,340.34,392.836,512.928,606.696,211.521,237.99,600.432,639.088,153.750,164.331,277.533,276.268\n12/29/2020,10:53,312.666,307.652,499.938,495.325,220.466,271.499,319.979,365.495,433.426,526.278,210.765,236.301,556.638,586.002,70.4508,79.6753,291.971,290.784\n12/29/2020,10:54,278.939,269.696,401.228,400.650,201.087,245.819,281.387,318.956,347.386,387.361,192.234,214.87,414.475,431.567,6.20667,11.4667,276.715,275.336\n12/29/2020,10:55,256.211,245.295,356.084,358.231,189.450,231.388,261.689,295.173,329.573,380.161,182.692,203.869,367.386,381.871,4.14766,8.10109,255.094,253.245\n12/29/2020,10:56,259.247,248.583,352.798,362.040,187.049,228.800,257.297,290.001,323.825,377.966,182.526,203.911,369.596,384.323,4.13778,7.79290,257.033,255.295\n12/29/2020,10:57,277.237,267.888,380.253,395.013,193.101,235.448,269.770,304.597,347.420,405.383,190.673,212.895,396.841,414.133,5.87585,9.94536,274.172,272.314\n12/29/2020,10:58,307.345,300.915,447.516,458.009,184.001,246.499,288.224,337.836,411.644,481.918,201.178,224.182,453.393,475.160,35.6057,43.5434,289.897,287.431\n12/29/2020,10:59,316.426,308.274,626.555,618.802,243.948,299.791,388.345,442.399,591.963,698.649,207.122,240.050,541.459,570.426,96.3541,105.808,275.029,272.657\n12/29/2020,11:00,264.669,251.592,415.285,430.600,200.153,236.922,300.187,332.577,411.735,465.543,184.477,202.812,410.209,424.539,43.8368,51.0624,245.953,243.855\n12/29/2020,11:01,230.229,214.506,306.892,316.560,173.717,204.425,232.958,256.333,294.162,335.347,166.229,182.197,324.913,334.618,5.58947,8.90827,225.977,224.609\n12/29/2020,11:02,246.337,232.496,316.403,327.598,178.843,211.943,228.637,252.489,296.755,340.435,175.207,193.284,331.700,343.063,4.95744,7.51896,241.950,240.476\n12/29/2020,11:03,298.642,290.070,409.451,429.010,202.787,243.895,272.297,304.863,384.382,449.333,203.912,227.219,436.197,459.087,19.4002,24.7289,287.249,285.503\n12/29/2020,11:04,344.955,340.781,573.332,596.216,237.299,288.720,355.399,403.108,549.356,648.800,236.182,265.643,553.215,583.957,29.1867,37.6829,329.971,327.714\n12/29/2020,11:05,359.123,354.628,574.788,597.65,233.716,283.320,355.324,400.639,546.424,643.411,236.463,265.506,558.922,587.406,68.7031,79.1032,328.324,326.215\n12/29/2020,11:06,334.943,326.399,595.848,614.777,233.950,282.583,361.281,405.879,575.460,671.719,233.659,261.462,547.301,572.956,50.7151,61.8785,310.135,308.593\n12/29/2020,11:07,320.182,309.901,522.152,537.625,218.806,262.004,325.723,361.983,499.602,577.479,218.556,243.264,534.995,559.680,37.3092,47.8435,299.242,298.101\n12/29/2020,11:08,343.803,336.238,524.860,545.799,220.853,266.168,319.466,356.443,505.951,589.124,228.092,255.881,642.350,679.216,70.5399,80.923,311.340,310.119\n12/29/2020,11:09,435.243,443.523,777.509,804.015,274.686,338.599,412.551,471.906,770.054,905.363,281.882,321.325,926.284,1001.12,269.608,284.400,341.992,340.435\n12/29/2020,11:10,486.404,492.436,1005.47,1022.58,326.261,405.824,511.950,593.983,1013.18,1177.45,316.878,363.067,1061.13,1128.19,338.159,349.958,338.631,337.759\n12/29/2020,11:11,661.833,689.119,1002.12,1028.02,326.662,406.625,504.396,585.494,1020.95,1188.27,322.939,371.148,1128.98,1208.84,810.941,822.050,311.358,310.772\n12/29/2020,11:12,629.068,649.94,1122.72,1140.41,342.581,425.066,529.763,617.617,1158.00,1344.26,336.481,386.203,1250.76,1339.84,851.276,855.032,269.674,269.055\n12/29/2020,11:13,604.355,622.183,1106.62,1126.36,333.665,413.065,507.665,592.438,1152.61,1341.07,330.031,378.415,1228.34,1316.75,862.504,865.936,239.321,238.800\n12/29/2020,11:14,611.507,631.222,1121.53,1143.40,339.827,421.503,510.294,597.658,1167.74,1361.47,337.917,388.654,1237.59,1329.41,859.635,863.073,246.496,245.800\n12/29/2020,11:15,638.529,662.717,1146.22,1168.39,353.573,439.600,526.675,619.213,1185.64,1382.11,351.797,406.083,1260.45,1355.51,853.858,857.707,275.494,274.383\n12/29/2020,11:16,657.216,683.019,1149.91,1168.89,363.905,453.056,540.231,635.154,1176.51,1368.22,357.897,412.662,1155.75,1224.14,808.732,814.799,306.025,305.257\n12/29/2020,11:17,446.698,435.069,639.516,643.342,278.434,333.679,377.681,426.110,619.865,696.156,263.660,295.652,642.583,680.452,211.615,220.369,324.623,323.539\n12/29/2020,11:18,405.075,400.784,488.155,501.799,254.507,305.024,327.016,367.182,469.012,533.967,241.239,270.161,587.785,606.343,178.834,186.722,330.878,328.985\n12/29/2020,11:19,350.084,336.539,428.184,438.506,243.933,290.980,303.816,339.195,409.875,467.019,224.965,251.231,463.259,478.525,36.3958,45.3584,321.336,319.702\n12/29/2020,11:20,326.320,312.718,422.920,433.033,237.792,282.195,287.396,319.994,411.141,470.874,221.271,246.701,566.611,603.130,38.4401,48.5040,303.735,302.484\n12/29/2020,11:21,339.791,335.297,571.807,598.813,254.628,306.721,323.959,369.295,599.764,715.488,256.478,291.708,1084.82,1166.68,210.821,217.210,285.42,284.067\n12/29/2020,11:22,510.63,521.990,1079.26,1097.95,341.101,422.031,473.653,559.555,1134.09,1370.76,332.038,382.955,1098.15,1157.23,563.260,574.873,272.303,270.510\n12/29/2020,11:23,279.234,258.661,806.629,798.740,291.891,350.942,399.601,459.033,823.181,950.339,251.070,280.701,433.884,446.680,21.9531,32.2186,256.116,255.094\n12/29/2020,11:24,346.713,338.425,505.154,521.298,233.688,277.000,294.858,329.793,513.743,588.178,219.694,245.059,476.146,490.996,137.150,157.468,252.710,251.123\n12/29/2020,11:25,453.363,451.829,850.407,880.659,297.004,364.126,395.969,460.855,885.792,1035.27,291.290,333.421,722.954,766.467,366.976,385.332,266.294,263.932\n12/29/2020,11:26,366.805,359.676,997.960,1014.50,321.678,394.430,440.429,514.565,1047.48,1204.98,317.641,364.079,687.681,725.536,187.282,199.304,274.584,272.689\n12/29/2020,11:27,434.448,446.953,979.783,997.415,315.365,386.114,419.355,487.058,1031.39,1186.54,327.081,376.113,1096.71,1168.23,448.635,458.301,274.197,272.066\n12/29/2020,11:28,333.886,326.540,1028.36,1039.12,320.547,391.916,419.473,485.798,1071.18,1228.82,312.632,355.847,758.549,803.324,175.738,184.517,265.528,263.577\n12/29/2020,11:29,463.616,476.370,997.257,1015.09,314.605,385.586,402.760,466.955,1052.42,1218.45,313.861,359.247,1052.08,1127.26,536.381,547.793,246.932,244.998\n12/29/2020,11:30,563.901,580.870,1068.43,1082.63,326.091,399.963,420.086,488.159,1136.30,1307.92,322.555,368.909,1188.04,1271.06,794.097,800.256,223.893,221.708\n12/29/2020,11:31,573.833,589.671,1075.18,1088.19,324.758,397.663,417.949,483.695,1146.6,1310.18,322.624,368.759,1182.51,1259.88,825.913,830.030,215.698,213.655\n12/29/2020,11:32,563.933,577.328,1022.99,1032.85,312.966,381.844,400.322,459.412,1084.02,1233.56,313.087,356.973,1083.31,1149.82,760.941,765.885,226.370,224.395\n12/29/2020,11:33,497.572,498.541,747.995,745.798,264.479,313.873,315.295,350.141,760.848,851.217,270.535,304.478,896.767,939.369,516.063,522.242,252.989,251.340\n12/29/2020,11:34,327.443,313.107,505.784,510.113,219.377,257.854,258.302,283.162,508.113,561.686,227.309,253.118,442.433,447.241,127.576,130.255,263.723,261.198\n12/29/2020,11:35,279.969,263.990,388.485,395.190,199.100,233.753,230.054,251.051,384.01,426.455,216.330,241.756,403.491,413.094,8.27069,8.35554,273.140,271.361\n12/29/2020,11:36,311.120,299.635,468.766,482.072,213.651,254.013,252.833,278.553,466.518,527.831,238.914,268.564,501.223,520.148,45.9851,50.7692,291.823,290.201\n12/29/2020,11:37,461.860,470.813,705.001,736.177,251.712,307.544,304.711,343.228,727.054,845.003,289.801,331.375,875.813,933.769,394.101,404.051,301.201,298.418\n12/29/2020,11:38,435.37,433.306,983.838,996.556,296.513,363.677,375.167,425.816,1026.80,1168.66,320.579,365.980,816.304,850.234,317.630,324.336,294.012,291.526\n12/29/2020,11:39,434.691,432.060,661.152,655.934,234.883,277.776,274.103,299.28,637.953,702.029,248.756,276.592,499.206,509.259,270.257,284.813,276.161,274.040\n12/29/2020,11:40,368.974,343.767,459.971,467.923,194.595,228.258,226.176,245.789,461.361,513.543,216.195,239.551,487.164,500.637,101.495,112.413,257.704,256.170\n12/29/2020,11:41,321.022,307.960,504.061,520.890,202.132,239.868,234.846,257.150,518.451,583.972,224.246,249.653,597.350,628.142,145.584,155.546,248.297,246.827\n12/29/2020,11:42,320.221,306.213,564.724,574.291,213.856,254.281,255.929,281.732,589.039,654.238,233.377,260.044,613.217,636.073,160.599,168.808,247.409,245.930\n12/29/2020,11:43,431.834,434.973,722.109,740.772,242.998,295.632,290.296,323.593,764.045,862.990,273.079,309.377,895.414,952.270,446.564,453.005,253.456,251.802\n12/29/2020,11:44,491.185,498.779,908.377,918.517,278.219,340.285,330.380,368.786,950.763,1064.41,308.689,350.685,983.87,1041.47,549.643,554.103,263.452,261.935\n12/29/2020,11:45,518.274,525.231,919.93,927.566,278.494,339.248,326.311,362.150,963.213,1070.00,313.523,355.582,985.234,1040.83,583.818,588.753,262.114,260.771\n12/29/2020,11:46,535.817,544.679,975.857,983.578,286.755,350.200,328.650,364.688,1036.31,1145.62,322.520,366.382,1067.22,1129.06,669.809,673.885,248.976,247.391\n12/29/2020,11:47,555.115,567.249,1004.59,1013.27,294.655,361.222,330.157,366.763,1097.32,1189.63,327.688,373.280,1099.70,1165.36,727.143,730.771,238.293,236.715\n12/29/2020,11:48,578.778,594.880,1035.90,1047.41,311.394,383.774,337.530,376.023,1147.58,1231.71,339.559,388.743,1130.89,1201.43,757.594,760.191,245.387,243.835\n12/29/2020,11:49,608.972,629.548,1073.25,1087.66,324.608,401.078,348.944,390.257,1183.58,1272.69,352.660,405.603,1166.22,1242.33,776.983,780.062,265.094,263.459\n12/29/2020,11:50,586.932,601.198,1079.95,1091.17,313.907,384.523,347.821,388.248,1188.20,1266.74,343.097,393.617,1164.51,1237.72,724.701,728.846,269.055,267.143\n12/29/2020,11:51,319.767,301.950,564.033,558.999,201.240,232.966,227.786,245.450,601.595,618.318,227.352,252.932,600.459,613.194,196.087,202.216,225.490,224.382\n12/29/2020,11:52,231.165,209.728,349.616,352.634,156.461,179.261,172.420,184.582,365.495,385.174,182.137,202.226,375.024,380.538,43.8174,48.9151,203.449,202.288\n12/29/2020,11:53,241.657,221.983,364.122,371.187,159.579,184.974,172.605,185.506,380.547,410.318,184.164,205.309,389.269,398.997,66.3432,75.5862,200.005,198.281\n12/29/2020,11:54,236.643,216.489,345.550,350.138,156.011,180.174,170.320,182.173,356.382,386.217,178.481,198.269,367.240,373.460,63.9484,70.8703,196.734,194.796\n12/29/2020,11:55,226.872,205.935,342.878,346.587,153.942,177.402,168.484,179.752,354.173,383.539,174.910,194.099,370.491,376.403,68.7527,75.1263,189.438,187.339\n12/29/2020,11:56,200.761,178.517,292.271,293.661,142.796,163.327,157.773,167.700,297.016,322.880,162.735,180.182,309.285,310.941,26.4563,31.0690,182.041,180.462\n12/29/2020,11:57,185.863,163.541,261.935,262.305,135.880,154.897,150.515,159.698,262.657,287.067,156.201,172.871,277.119,277.918,9.57422,12.2153,176.177,174.339\n12/29/2020,11:58,182.619,161.244,255.291,255.72,133.851,152.692,147.811,156.624,254.572,278.791,155.945,172.587,269.315,269.876,8.65580,10.7526,174.196,172.192\n12/29/2020,11:59,186.125,165.024,258.773,259.113,133.48,152.634,147.955,156.893,256.37,280.999,158.121,175.103,270.851,271.165,12.6158,14.764,175.388,173.795\n12/29/2020,12:00,183.124,162.599,254.015,254.373,131.319,150.581,147.591,156.792,249.419,274.988,158.899,176.204,268.178,268.986,1.43194,2.31391,178.784,177.378\n12/29/2020,12:01,184.195,164.466,256.724,258.513,131.284,151.083,148.31,157.945,251.310,279.637,160.831,178.381,271.940,274.231,2.60712,3.75706,178.824,177.404\n12/29/2020,12:02,183.399,163.878,257.303,259.518,130.834,150.625,147.567,156.880,251.030,281.407,160.958,178.446,273.712,276.552,3.49592,4.68655,177.242,176.008\n12/29/2020,12:03,180.738,161.104,254.146,255.783,129.519,148.939,146.449,155.173,246.385,277.933,158.760,175.557,269.859,272.508,0.237011,-0.176114,176.515,175.077\n12/29/2020,12:04,177.643,157.850,248.854,250.116,127.989,147.099,144.793,152.961,241.142,272.171,155.589,171.455,264.694,266.877,-0.651783,-2.06444,174.153,172.706\n12/29/2020,12:05,175.300,155.447,245.764,246.966,127.028,145.926,143.680,151.418,239.428,269.671,152.976,167.952,261.069,263.261,0.799914,-0.498986,170.651,169.034\n12/29/2020,12:06,172.094,151.797,240.020,240.486,125.427,143.779,142.214,149.324,234.466,262.592,150.561,164.652,254.2,255.631,-0.607343,-2.62213,168.134,166.782\n12/29/2020,12:07,178.441,158.570,247.168,248.452,126.657,145.670,143.770,150.841,242.738,271.518,152.173,166.253,262.114,264.721,4.89823,3.41461,170.446,168.583\n12/29/2020,12:08,219.188,200.489,327.156,331.252,143.012,166.598,160.048,169.132,329.178,366.143,166.498,183.052,325.137,330.358,79.9664,82.3372,171.538,170.059\n12/29/2020,12:09,185.643,164.883,266.151,267.248,131.218,150.932,149.573,156.635,263.978,292.654,156.223,170.003,281.111,284.115,14.2946,13.9079,172.177,170.821\n12/29/2020,12:10,220.587,203.686,330.644,338.384,143.758,168.435,161.520,171.040,339.565,381.750,171.175,189.348,389.997,404.139,101.006,103.079,172.435,171.078\n12/29/2020,12:11,229.759,210.903,387.758,387.06,154.565,180.381,171.628,181.884,395.940,435.908,178.892,197.621,403.086,409.170,111.152,114.184,172.798,171.111\n12/29/2020,12:12,191.569,170.146,274.911,276.685,135.008,155.383,152.682,159.844,274.611,304.110,158.927,172.527,292.649,295.668,20.4026,22.6401,172.358,171.040\n12/29/2020,12:13,227.827,209.767,308.486,313.216,142.944,166.253,159.282,167.545,311.470,346.772,167.293,182.595,321.307,328.289,83.1316,87.4690,172.675,171.642\n12/29/2020,12:14,400.442,397.565,677.955,698.578,213.135,261.332,224.447,245.787,720.586,820.794,246.406,280.705,730.131,772.673,472.916,481.633,175.696,174.651\n12/29/2020,12:15,385.759,378.625,677.037,684.223,213.210,257.950,224.382,243.926,710.985,793.563,242.777,273.255,622.615,642.448,413.072,420.342,178.025,177.125\n12/29/2020,12:16,386.356,381.681,678.455,689.854,213.037,258.689,223.797,244.004,719.607,808.332,248.544,281.238,708.883,748.073,432.985,440.941,180.074,179.042\n12/29/2020,12:17,373.526,366.473,691.649,699.380,215.217,260.668,225.599,246.069,735.986,822.776,252.898,285.965,742.325,779.666,417.440,424.259,179.002,178.255\n12/29/2020,12:18,405.481,404.388,715.496,732.145,221.190,270.272,230.140,252.516,768.904,868.820,262.791,299.702,802.026,852.184,495.565,503.934,175.991,175.666\n12/29/2020,12:19,480.588,486.206,914.982,928.611,258.370,318.673,265.112,294.139,986.734,1112.99,308.966,356.100,1018.31,1082.53,685.803,689.706,174.822,174.296\n12/29/2020,12:20,487.705,493.493,935.232,947.280,262.459,323.552,267.960,297.675,1010.33,1134.77,315.761,364.634,1037.33,1101.97,715.395,716.935,170.734,170.159\n12/29/2020,12:21,480.775,485.351,925.539,936.238,261.016,321.231,265.291,294.777,1002.59,1120.70,316.532,366.073,1034.39,1098.57,713.638,713.859,166.996,166.376\n12/29/2020,12:22,478.380,482.181,924.131,935.260,261.454,321.644,264.005,293.763,1003.96,1118.85,318.795,369.930,1031.36,1094.04,714.349,714.798,164.425,164.051\n12/29/2020,12:23,451.568,451.875,863.504,872.855,251.145,307.381,252.560,280.003,938.950,1041.11,306.539,355.311,960.698,1016.83,655.685,657.098,162.329,161.927\n12/29/2020,12:24,432.749,430.993,821.895,830.625,244.392,298.178,244.703,270.654,894.885,989.149,299.452,347.046,919.008,971.006,615.883,617.982,161.011,160.374\n12/29/2020,12:25,433.402,432.337,820.810,831.327,245.178,299.536,244.669,271.084,895.480,991.165,302.225,351.595,922.317,975.769,618.121,621.183,161.323,160.462\n12/29/2020,12:26,426.921,424.937,807.163,817.057,243.868,297.613,243.148,269.450,879.961,972.559,300.698,350.305,897.573,948.664,598.248,601.808,162.797,161.602\n12/29/2020,12:27,405.042,400.060,765.969,773.375,237.217,288.367,236.867,261.936,833.798,918.073,291.855,339.526,846.981,891.499,544.854,548.394,164.376,163.106\n12/29/2020,12:28,399.216,394.973,729.136,739.440,231.982,282.146,231.085,255.527,794.306,876.352,285.757,333.057,809.293,854.155,522.693,527.954,166.939,165.769\n12/29/2020,12:29,412.534,409.218,771.445,780.346,240.539,293.381,238.549,264.584,838.643,926.790,298.656,350.138,858.599,905.391,550.814,556.001,169.989,168.817\n12/29/2020,12:30,383.698,377.316,709.292,715.980,230.257,279.400,227.998,251.713,768.816,847.115,284.070,332.025,788.508,828.464,477.526,483.881,172.001,170.989\n12/29/2020,12:31,381.835,375.961,682.553,691.538,226.799,275.293,223.942,246.808,739.247,817.856,280.111,327.911,766.792,808.043,461.398,470.397,175.195,174.132\n12/29/2020,12:32,405.815,402.375,736.866,747.033,238.153,289.899,234.334,259.188,797.317,885.146,297.396,350.547,823.095,868.938,510.962,520.05,178.337,177.138\n12/29/2020,12:33,418.221,415.659,764.071,773.483,244.554,297.849,240.176,265.955,826.352,916.906,307.831,364.164,853.146,900.948,539.08,547.487,179.812,178.430\n12/29/2020,12:34,428.435,427.570,782.534,792.901,249.437,304.014,244.676,271.280,847.073,940.536,316.489,376.069,877.691,928.114,564.267,572.612,180.185,178.761\n12/29/2020,12:35,437.394,437.084,800.026,809.615,254.373,310.038,249.051,276.487,865.404,959.718,324.312,386.577,894.398,944.573,582.576,590.487,181.045,179.673\n12/29/2020,12:36,452.319,453.438,828.903,838.946,261.037,318.738,256.015,284.984,895.617,994.244,334.688,400.794,926.824,979.765,613.046,620.02,183.863,182.421\n12/29/2020,12:37,447.618,447.823,809.576,819.623,258.964,315.710,256.304,285.314,872.452,966.152,329.616,394.398,892.155,942.331,587.923,595.252,187.397,186.007\n12/29/2020,12:38,467.892,470.840,848.432,861.122,267.182,327.259,267.101,298.478,915.434,1017.49,342.282,411.810,941.995,998.591,635.572,643.124,189.257,187.787\n12/29/2020,12:39,444.746,443.164,815.803,822.599,262.252,319.039,265.945,296.061,877.908,967.419,332.779,397.836,907.669,956.484,579.647,585.536,191.569,190.325\n12/29/2020,12:40,356.109,346.617,589.199,602.703,224.026,269.866,232.888,256.210,634.953,696.670,269.725,316.369,688.779,723.049,362.444,370.466,194.330,193.331\n12/29/2020,12:41,460.431,460.57,783.61,796.325,262.051,319.052,270.368,300.604,832.174,919.077,326.131,388.782,778.305,820.024,547.208,560.236,203.778,202.852\n12/29/2020,12:42,518.655,526.65,924.315,945.191,289.443,355.946,298.535,335.903,991.02,1107.14,377.498,456.427,1021.62,1089.99,693.621,702.310,215.664,214.546\n12/29/2020,12:43,518.786,526.904,928.948,943.972,290.643,356.859,301.498,339.570,992.438,1103.44,380.026,457.873,1026.73,1092.62,684.133,689.637,223.046,221.468\n12/29/2020,12:44,527.079,536.668,942.350,957.950,293.708,361.171,306.282,345.998,1006.27,1119.46,386.342,465.400,1035.75,1102.19,688.583,695.063,228.591,227.239\n12/29/2020,12:45,539.751,551.288,956.182,974.140,296.580,365.293,310.373,351.383,1021.64,1139.57,393.755,474.834,1055.56,1128.06,709.673,716.823,232.139,230.504\n12/29/2020,12:46,543.456,554.944,944.377,961.567,295.006,362.952,308.558,349.076,1006.49,1120.44,392.468,471.956,1038.82,1108.94,712.283,719.429,233.246,231.568\n12/29/2020,12:47,573.878,590.221,1025.24,1047.49,309.579,382.468,323.619,367.849,1096.13,1227.96,425.130,514.991,1139.66,1221.84,791.010,798.293,233.771,231.958\n12/29/2020,12:48,589.610,607.977,1067.51,1089.05,316.844,391.599,331.331,377.260,1139.04,1275.43,443.548,537.896,1179.19,1264.62,819.242,825.711,238.648,236.698\n12/29/2020,12:49,585.687,603.080,1059.05,1080.27,315.358,389.496,330.140,375.839,1131.12,1264.38,444.346,538.084,1171.18,1256.97,820.903,826.231,235.238,233.449\n12/29/2020,12:50,577.769,594.319,1049.35,1068.81,312.910,385.822,326.862,371.559,1124.54,1253.88,445.108,538.444,1165.37,1249.70,827.598,831.675,226.298,224.971\n12/29/2020,12:51,573.228,589.425,1044.02,1064.12,311.543,384.231,323.141,367.047,1122.76,1251.57,447.828,541.874,1160.58,1245.71,834.852,838.915,219.062,217.905\n12/29/2020,12:52,570.539,586.125,1041.66,1061.37,310.774,382.992,320.386,363.642,1122.58,1249.84,452.046,546.446,1160.96,1244.67,847.087,851.336,211.699,210.637\n12/29/2020,12:53,572.297,588.125,1044.35,1063.92,311.122,383.366,319.138,362.019,1128.06,1255.83,459.559,555.338,1165.19,1249.83,866.902,870.899,205.349,203.856\n12/29/2020,12:54,571.315,587.299,1044.24,1062.87,311.364,383.452,316.962,359.376,1128.56,1254.60,467.694,565.126,1169.36,1253.44,874.669,878.550,202.253,200.480\n12/29/2020,12:55,530.393,539.012,981.837,993.296,300.714,368.144,304.887,343.904,1055.20,1162.29,451.531,541.446,1089.17,1160.35,771.145,775.178,204.551,202.970\n12/29/2020,12:56,398.682,391.319,606.347,611.311,236.323,284.209,242.720,267.313,638.566,681.274,319.947,370.856,622.886,646.592,391.472,402.376,208.639,207.766\n12/29/2020,12:57,530.578,542.118,855.298,883.759,282.182,348.617,282.440,318.509,912.431,1016.38,422.542,506.749,885.863,941.343,682.639,697.706,220.297,218.612\n12/29/2020,12:58,567.518,584.905,957.489,982.173,300.746,372.103,298.613,338.242,1021.75,1137.88,463.972,559.430,1048.03,1127.17,778.342,793.178,226.873,225.039\n12/29/2020,12:59,491.199,496.250,793.963,806.107,271.259,331.415,270.326,302.468,835.613,912.875,401.180,475.347,833.849,882.085,588.743,602.507,226.722,224.811\n12/29/2020,13:00,574.532,593.515,988.599,1017.67,306.726,381.293,304.424,346.318,1055.5,1181.32,489.034,591.733,1144.34,1229.49,804.743,815.157,236.355,234.153\n12/29/2020,13:01,601.689,624.092,1041.01,1065.46,318.825,396.356,317.878,362.367,1100.40,1228.83,516.491,624.865,1179.26,1267.51,836.680,843.481,252.770,250.85\n12/29/2020,13:02,628.852,654.377,1060.78,1085.83,325.445,404.887,327.012,373.304,1107.54,1236.97,530.677,642.326,1150.83,1237.09,826.192,833.638,276.217,274.841\n12/29/2020,13:03,537.223,547.906,917.940,924.511,300.385,368.128,308.911,349.100,932.967,1019.23,472.236,561.500,952.298,1001.72,553.275,562.852,296.480,296.028\n12/29/2020,13:04,598.728,623.13,895.817,942.422,299.789,375.709,311.729,356.557,923.501,1043.56,482.249,584.816,1025.31,1118.77,660.608,675.101,314.020,312.743\n12/29/2020,13:05,549.217,571.632,840.260,872.512,287.760,357.831,305.114,347.405,858.759,957.117,457.521,549.658,977.459,1054.31,547.721,558.147,322.200,320.856\n12/29/2020,13:06,685.276,725.496,1173.46,1208.69,344.664,432.946,361.097,418.088,1212.32,1366.37,587.769,717.702,1301.71,1410.74,859.054,866.616,330.974,328.790\n12/29/2020,13:07,658.760,696.033,1097.97,1134.22,330.218,414.342,350.126,404.721,1129.29,1269.69,557.820,679.250,1201.81,1300.89,779.063,789.161,330.620,328.392\n12/29/2020,13:08,638.979,677.901,1076.29,1113.79,322.899,405.859,345.515,400.237,1109.62,1248.50,551.267,672.139,1198.95,1305.67,762.733,771.452,325.524,322.970\n12/29/2020,13:09,631.932,668.843,1099.82,1138.93,322.688,406.217,346.533,402.869,1137.69,1280.46,562.921,687.144,1226.45,1334.51,775.156,782.757,316.298,313.905\n12/29/2020,13:10,464.094,481.646,711.552,740.924,251.525,313.765,279.012,319.492,719.777,794.894,403.589,483.077,813.128,874.584,391.190,398.961,303.866,302.039\n12/29/2020,13:11,564.634,595.893,945.564,983.148,289.943,365.827,314.028,364.820,972.390,1090.38,502.080,610.936,1055.34,1149.54,645.649,657.013,297.865,296.127\n12/29/2020,13:12,443.323,457.329,700.629,740.008,245.561,309.200,272.840,314.202,713.167,793.126,399.895,481.151,775.746,839.494,375.019,386.413,288.635,287.654\n12/29/2020,13:13,331.306,332.567,462.849,478.563,202.008,250.825,230.665,261.619,444.244,476.943,290.878,340.756,495.923,519.543,105.726,115.803,283.833,283.302\n12/29/2020,13:14,329.301,332.498,431.599,459.724,196.617,247.031,224.348,255.782,414.524,458.675,281.389,333.181,465.909,497.864,102.408,115.583,282.264,281.493\n12/29/2020,13:15,293.834,293.686,348.437,368.115,181.739,227.719,208.708,236.703,318.492,351.004,240.989,282.916,348.288,367.738,16.5759,25.6340,281.293,280.491\n12/29/2020,13:16,291.762,291.896,339.128,359.901,180.367,226.488,205.899,233.783,307.664,343.714,238.168,279.858,339.141,359.276,11.1345,19.1619,281.819,280.990\n12/29/2020,13:17,303.209,304.910,358.799,380.602,184.781,232.351,208.228,236.951,328.601,369.601,248.453,292.305,365.643,388.689,22.264,33.8230,287.237,285.855\n12/29/2020,13:18,427.573,446.461,602.825,641.663,228.848,291.463,248.201,287.739,599.452,687.38,363.798,439.006,660.267,720.032,308.521,326.935,293.631,291.839\n12/29/2020,13:19,454.644,473.765,710.541,740.826,247.254,312.974,265.097,307.920,712.491,804.541,408.570,490.890,759.639,819.221,381.535,396.410,294.277,292.564\n12/29/2020,13:20,471.644,493.434,730.002,765.372,251.493,319.142,268.090,312.220,737.659,836.533,422.389,509.385,793.846,859.518,423.451,437.356,295.715,293.775\n12/29/2020,13:21,471.296,490.983,740.888,770.119,254.460,322.032,270.704,314.837,746.459,841.343,427.316,513.747,807.415,871.006,423.729,435.103,297.095,295.456\n12/29/2020,13:22,446.274,461.881,682.590,707.960,244.420,308.014,261.628,302.853,681.459,764.677,401.735,480.882,746.408,801.314,361.312,371.488,297.447,295.752\n12/29/2020,13:23,423.875,436.766,619.215,645.768,234.073,294.600,252.484,291.273,610.935,686.106,374.049,446.957,661.185,708.968,293.976,304.658,299.241,297.637\n12/29/2020,13:24,405.586,416.926,579.230,605.653,226.958,285.228,246.552,283.811,567.370,637.371,358.457,427.874,633.129,679.947,251.166,261.633,300.707,298.969\n12/29/2020,13:25,419.565,433.126,604.191,631.602,230.336,289.774,250.043,288.296,591.849,665.875,371.860,444.778,644.624,692.496,276.294,286.890,303.036,301.129\n12/29/2020,13:26,418.353,431.962,600.97,627.763,228.880,287.713,249.185,287.189,586.597,659.561,372.552,445.212,643.526,690.842,270.818,281.030,304.911,303.042\n12/29/2020,13:27,422.745,437.578,611.699,638.089,229.484,288.528,250.057,288.305,597.484,672.216,379.489,453.524,666.154,715.267,280.310,290.144,306.706,304.723\n12/29/2020,13:28,438.186,455.138,643.820,672.963,233.708,294.387,254.072,293.501,631.746,713.279,397.111,475.542,700.158,754.500,315.180,324.780,307.987,305.949\n12/29/2020,13:29,422.683,437.4,615.568,642.104,227.216,285.614,247.667,285.387,599.004,674.139,382.749,456.478,663.418,711.854,277.722,285.976,307.830,305.908\n12/29/2020,13:30,402.065,413.852,571.660,596.819,218.380,274.217,239.061,274.649,549.682,618.387,360.283,428.498,612.523,656.443,224.493,231.445,308.182,306.295\n12/29/2020,13:31,399.311,411.125,553.428,580.994,214.397,269.333,234.999,269.592,529.438,598.050,351.741,418.498,594.385,639.024,213.003,220.007,309.382,307.495\n12/29/2020,13:32,408.534,421.651,574.910,602.973,216.729,272.519,237.173,272.415,551.490,623.532,361.812,430.639,612.532,659.117,234.363,241.302,310.435,308.649\n12/29/2020,13:33,439.743,457.465,642.883,674.082,226.828,286.086,247.014,284.918,626.561,710.711,397.099,474.576,694.758,751.791,313.726,321.020,311.771,309.840\n12/29/2020,13:34,427.899,443.381,630.996,656.73,222.545,279.767,243.921,280.523,611.487,688.163,386.839,459.931,674.231,725.065,288.149,293.860,310.921,308.925\n12/29/2020,13:35,413.999,427.901,597.101,623.579,214.539,269.953,237.759,273.018,574.956,648.493,367.022,436.632,631.650,680.190,259.007,263.990,308.262,306.396\n12/29/2020,13:36,417.263,431.357,611.535,638.468,214.040,269.490,239.249,274.770,591.163,667.489,372.948,443.934,649.512,700.071,278.708,283.524,304.615,302.792\n12/29/2020,13:37,387.718,397.579,562.465,585.216,202.563,254.131,230.659,263.691,537.603,603.729,343.300,406.558,592.270,634.9,220.724,224.492,298.442,296.534\n12/29/2020,13:38,369.935,378.883,516.253,541.674,192.084,241.382,222.528,253.914,490.457,554.674,318.728,378.345,541.411,583.269,192.273,196.671,290.883,289.167\n12/29/2020,13:39,371.559,380.267,540.677,564.778,193.041,242.115,224.693,256.280,518.641,585.836,330.561,391.912,569.308,612.688,220.378,224.834,282.818,281.076\n12/29/2020,13:40,375.838,385.907,556.788,583.054,192.820,242.262,224.927,256.781,541.318,613.565,341.277,405.536,600.294,648.270,259.069,263.989,273.376,271.616\n12/29/2020,13:41,368.339,378.050,557.424,582.263,190.117,238.647,222.139,253.276,545.112,617.037,341.906,405.971,599.933,647.295,267.478,272.006,263.602,261.885\n12/29/2020,13:42,336.943,343.116,502.328,523.475,178.253,222.661,210.347,238.476,487.648,549.332,311.897,368.772,545.288,586.391,213.440,216.732,254.550,252.813\n12/29/2020,13:43,300.369,302.270,423.694,441.645,163.050,203.012,194.833,219.547,403.313,452.583,267.745,315.315,450.375,482.134,135.080,137.610,247.113,245.330\n12/29/2020,13:44,293.863,295.152,410.093,428.593,159.094,198.296,189.486,213.253,390.132,439.700,260.971,307.816,430.156,460.726,132.443,134.719,241.099,239.141\n12/29/2020,13:45,264.160,261.726,347.605,362.202,147.053,182.534,176.771,197.470,321.550,361.296,224.816,263.876,348.044,370.515,65.8635,66.9413,235.372,233.353\n12/29/2020,13:46,257.848,254.751,332.111,347.088,143.110,177.763,171.478,191.156,305.517,345.065,218.532,256.322,337.343,359.805,59.3802,60.6941,231.585,229.604\n12/29/2020,13:47,259.204,256.069,331.408,347.099,141.937,176.636,168.908,188.062,304.446,345.339,219.946,257.855,335.932,358.604,65.6856,66.9314,229.937,227.975\n12/29/2020,13:48,271.691,269.331,363.402,379.408,146.232,181.966,171.777,191.335,338.292,383.231,239.793,280.478,374.792,399.518,99.5926,101.214,229.870,227.955\n12/29/2020,13:49,264.800,261.123,345.088,360.125,142.201,176.549,167.416,185.866,316.601,358.262,228.970,266.995,352.751,375.462,79.2298,80.2964,230.331,228.527\n12/29/2020,13:50,264.400,260.553,339.837,354.924,140.242,174.073,165.380,183.480,309.508,350.649,225.813,262.980,348.774,371.716,76.7215,77.6499,231.041,229.262\n12/29/2020,13:51,270.226,266.882,356.505,371.543,141.468,175.509,166.931,185.331,326.735,369.458,234.693,273.084,369.587,393.595,94.2602,94.9674,231.178,229.369\n12/29/2020,13:52,259.500,254.758,335.466,349.010,136.043,168.331,162.856,180.350,302.426,341.249,219.390,254.449,338.283,358.852,67.9178,67.8563,230.031,228.290\n12/29/2020,13:53,249.773,243.962,314.039,327.110,130.42,161.014,158.542,175.210,279.153,315.038,204.600,236.863,312.950,331.855,48.6065,47.4569,227.654,225.984\n12/29/2020,13:54,243.050,236.471,303.849,316.253,126.398,155.605,155.723,171.832,268.311,302.468,196.347,226.761,298.150,315.610,40.0792,38.5878,224.172,222.548\n12/29/2020,13:55,237.771,230.863,296.381,308.721,122.614,150.708,153.144,168.795,261.354,294.979,190.521,219.920,289.200,306.433,38.4843,37.0273,219.563,217.912\n12/29/2020,13:56,230.250,222.326,289.028,300.256,118.689,145.281,150.291,165.257,254.658,286.725,184.886,212.752,280.545,296.328,34.223,32.2330,213.787,212.232\n12/29/2020,13:57,223.981,215.412,279.244,290.855,114.543,139.828,146.820,161.217,246.367,278.283,179.348,206.351,272.430,288.268,34.1292,32.3651,207.345,205.806\n12/29/2020,13:58,220.432,211.488,282.076,293.130,112.479,136.754,144.879,158.796,251.729,283.749,181.519,208.376,278.680,294.561,43.8317,42.0072,200.678,199.232\n12/29/2020,13:59,211.757,201.703,270.601,280.770,108.395,131.085,140.494,153.451,241.125,271.708,174.836,200.127,265.390,279.835,37.8572,35.0313,194.227,192.816\n12/29/2020,14:00,202.749,191.517,255.022,263.929,104.086,124.979,135.321,147.065,225.653,253.724,165.918,189.126,248.021,260.625,27.6510,24.2005,188.570,187.154\n12/29/2020,14:01,207.383,196.636,267.942,277.812,104.655,125.358,134.483,145.945,241.764,272.257,176.405,201.308,266.824,281.810,52.4335,49.8933,184.103,182.662\n12/29/2020,14:02,202.470,190.706,265.423,273.897,102.502,122.034,131.046,141.639,239.851,268.623,175.522,199.354,266.438,279.871,50.0928,47.4862,180.257,178.856\n12/29/2020,14:03,192.223,178.819,245.348,252.263,97.4921,115.250,124.931,134.320,217.939,243.210,162.725,183.698,241.718,252.173,29.5965,26.4068,176.847,175.460\n12/29/2020,14:04,186.705,172.460,232.390,239.073,93.6772,110.108,119.923,128.478,204.431,228.433,155.101,174.677,227.819,237.820,23.1034,20.0375,173.177,171.825\n12/29/2020,14:05,184.515,169.810,233.709,239.967,91.7720,107.387,117.090,125.120,206.832,230.665,156.330,175.647,231.517,241.398,29.6411,26.3531,168.855,167.523\n12/29/2020,14:06,177.792,162.436,223.659,229.227,87.9749,102.116,112.725,119.911,197.060,219.178,149.972,167.855,221.100,230.165,24.5848,20.9181,163.915,162.664\n12/29/2020,14:07,170.623,154.501,214.547,219.151,84.1381,96.8798,108.655,114.888,188.411,208.643,144.019,160.448,211.486,219.305,19.1830,15.5467,158.611,157.412\n12/29/2020,14:08,166.416,149.925,211.086,215.629,81.2052,93.0655,105.562,111.087,186.629,206.592,142.625,158.608,211.006,218.693,22.9011,19.3184,153.024,151.908\n12/29/2020,14:09,159.511,142.317,202.821,206.346,77.5627,88.1657,101.762,106.373,179.424,197.720,137.603,152.198,203.814,210.469,18.6398,15.2630,147.417,146.386\n12/29/2020,14:10,155.327,137.747,201.476,204.573,75.2118,85.0225,99.1780,103.051,180.146,197.998,137.523,151.583,204.265,210.394,21.8197,18.5797,142.092,141.010\n12/29/2020,14:11,147.213,128.998,190.712,192.760,71.5512,80.0319,95.1226,98.0442,170.598,186.559,130.617,142.995,193.381,198.268,13.4009,9.86223,136.942,135.829\n12/29/2020,14:12,143.759,125.272,189.541,191.488,69.6415,77.4301,92.3990,94.8102,172.198,188.130,131.090,143.147,196.963,201.782,17.8745,14.6466,131.916,130.78\n12/29/2020,14:13,141.891,123.186,192.643,194.475,68.6535,75.8748,90.3288,92.3888,178.588,194.757,134.432,146.534,204.898,209.935,26.3230,23.2566,127.11,125.964\n12/29/2020,14:14,142.396,123.468,201.600,203.606,68.8257,75.8714,89.3613,91.1997,191.473,208.593,141.908,154.780,217.298,222.757,40.2522,37.6243,122.688,121.534\n12/29/2020,14:15,146.670,128.068,219.504,221.979,70.6521,77.9612,89.9255,91.7310,214.577,233.939,156.302,171.068,243.425,250.608,65.1187,63.1949,118.764,117.589\n12/29/2020,14:16,146.363,127.708,226.555,228.648,70.9567,77.8461,89.0180,90.4404,225.416,244.823,162.592,177.683,256.478,263.700,75.8386,74.3438,114.985,113.855\n12/29/2020,14:17,148.952,130.338,243.652,245.286,72.7520,79.7806,89.5221,90.8372,247.195,267.947,176.256,192.726,283.642,292.006,97.7966,96.6611,111.168,110.020\n12/29/2020,14:18,153.363,135.613,250.970,254.058,73.4184,80.7409,89.0487,90.1299,258.943,281.93,184.659,202.849,300.405,311.458,119.888,120.221,107.723,106.600\n12/29/2020,14:19,201.667,189.281,371.792,381.85,92.6533,105.973,105.366,109.882,398.914,445.868,280.864,317.559,474.726,504.395,280.872,283.814,105.903,104.766\n12/29/2020,14:20,189.878,174.950,366.399,370.538,91.0259,102.459,103.072,106.484,393.168,431.863,275.878,307.742,467.996,490.640,255.310,257.524,103.161,102.143\n12/29/2020,14:21,214.445,202.867,419.647,430.184,99.6090,114.216,109.951,115.171,457.265,510.639,322.411,365.329,546.418,581.015,341.226,343.770,100.984,99.9765\n12/29/2020,14:22,257.123,250.631,530.162,544.296,117.204,136.807,124.693,133.147,584.150,658.564,411.524,472.310,701.395,752.528,487.272,489.305,99.7755,98.8329\n12/29/2020,14:23,279.948,276.139,600.848,616.281,128.177,150.643,133.837,144.206,666.124,752.653,468.997,541.090,798.636,859.851,575.117,576.200,97.6748,96.7477\n12/29/2020,14:24,293.848,291.788,642.062,659.774,134.374,158.719,138.950,150.359,716.733,811.334,505.786,586.038,863.722,933.373,638.369,639.106,94.0361,93.1631\n12/29/2020,14:25,307.862,307.850,684.685,703.632,140.690,166.857,144.233,156.559,767.890,869.798,543.349,631.852,926.934,1004.78,702.954,703.083,90.2895,89.4015\n12/29/2020,14:26,311.862,312.438,703.841,723.107,143.223,170.038,146.105,158.539,792.490,896.307,563.152,655.933,959.597,1041.41,735.637,735.224,86.4735,85.6331\n12/29/2020,14:27,305.803,305.348,696.543,715.101,141.825,167.855,144.538,156.177,786.319,886.527,561.706,653.071,951.946,1031.93,731.038,730.085,83.1980,82.3382\n12/29/2020,14:28,299.246,298.305,678.794,698.292,138.950,164.146,141.536,152.331,768.451,865.321,552.578,642.364,928.753,1007.33,722.606,721.912,80.1191,79.3272\n12/29/2020,14:29,306.191,306.057,701.338,721.505,142.246,168.524,144.270,155.461,794.619,895.831,575.583,670.501,964.366,1047.62,758.549,758.393,77.7172,77.0246\n12/29/2020,14:30,300.704,299.803,690.930,710.587,140.288,165.862,142.390,152.909,783.477,881.201,571.674,665.033,954.008,1035.55,752.252,752.541,75.4393,74.7715\n12/29/2020,14:31,305.121,304.907,701.91,723.501,141.885,168.309,143.613,154.446,797.140,898.195,586.156,683.569,972.577,1058.14,776.531,777.270,74.1069,73.3745\n12/29/2020,14:32,301.031,300.568,693.583,714.282,140.311,166.206,141.868,152.301,788.150,886.506,583.254,680.004,964.120,1049.07,771.889,772.710,72.6873,72.0043\n12/29/2020,14:33,301.057,300.930,696.661,717.880,140.466,166.693,141.788,152.295,792.724,891.876,590.550,689.487,973.525,1061.01,781.712,783.035,71.3802,70.7706\n12/29/2020,14:34,296.032,295.264,687.291,708.029,138.627,164.478,139.808,149.994,782.474,879.310,586.549,684.745,962.807,1050.00,773.515,775.124,70.1497,69.5223\n12/29/2020,14:35,294.168,293.699,683.276,704.562,137.634,163.72,138.621,148.916,778.651,875.270,587.667,686.952,961.343,1049.90,774.971,776.708,69.326,68.7943\n12/29/2020,14:36,292.421,292.030,680.078,701.418,136.903,163.212,137.628,147.878,775.343,872.341,588.946,689.038,958.857,1048.30,776.210,777.862,68.7395,68.2301\n12/29/2020,14:37,293.365,293.128,683.068,704.557,137.031,163.504,137.564,147.882,778.941,876.645,595.684,697.343,967.294,1058.46,786.020,787.934,68.4635,67.9029\n12/29/2020,14:38,289.490,288.940,676.369,697.664,135.536,161.823,136.142,146.299,771.859,868.122,593.710,694.956,962.865,1053.38,778.594,781.227,68.1408,67.6094\n12/29/2020,14:39,282.615,281.662,664.186,685.744,133.234,159.225,133.853,143.708,758.202,852.592,586.736,686.797,947.853,1037.95,760.896,764.192,67.7428,67.1400\n12/29/2020,14:40,277.115,275.492,652.967,675.077,131.112,156.987,131.757,141.447,746.132,839.569,581.244,680.614,936.250,1026.25,747.056,750.729,67.6008,66.9639\n12/29/2020,14:41,270.945,268.853,637.814,659.933,128.43,153.880,129.088,138.366,728.872,820.325,571.564,668.880,916.885,1004.89,728.154,731.660,67.6453,67.0228\n12/29/2020,14:42,267.548,265.136,626.873,648.723,126.362,151.567,127.033,136.047,716.451,806.94,565.947,662.662,901.893,989.578,716.896,720.932,67.8319,67.1417\n12/29/2020,14:43,267.351,265.246,626.407,648.568,126.111,151.525,126.470,135.471,715.993,806.997,570.282,668.240,910.872,1000.44,721.944,726.515,68.3118,67.6591\n12/29/2020,14:44,271.839,270.521,636.326,659.225,127.607,153.961,127.728,137.077,728.056,822.593,583.471,684.825,927.022,1019.50,741.226,746.093,68.8389,68.0784\n12/29/2020,14:45,273.020,272.137,641.352,664.688,128.163,154.817,128.212,137.69,734.260,830.811,591.958,695.644,935.504,1029.41,749.146,753.910,69.2434,68.4110\n12/29/2020,14:46,270.524,269.495,637.568,660.868,127.149,153.926,127.260,136.613,729.914,826.102,592.286,696.019,932.251,1026.10,742.312,747.297,69.3588,68.6159\n12/29/2020,14:47,267.015,266.067,632.200,655.493,125.733,152.555,126.083,135.344,724.064,819.137,591.615,695.346,929.401,1023.29,737.05,741.971,69.2195,68.4218\n12/29/2020,14:48,263.657,262.614,626.193,649.384,124.295,150.999,124.691,133.922,717.548,811.817,590.295,693.890,924.836,1018.92,731.418,736.225,68.8553,67.9813\n12/29/2020,14:49,260.517,259.398,621.247,644.834,122.875,149.743,123.403,132.696,712.242,806.388,589.773,693.799,921.438,1015.6,727.800,732.229,68.4279,67.5128\n12/29/2020,14:50,257.556,256.584,615.991,639.274,121.485,148.339,122.091,131.431,706.390,800.112,588.558,692.826,915.792,1010.34,723.510,728.057,67.9771,67.0822\n12/29/2020,14:51,254.035,253.075,607.982,630.619,119.822,146.625,120.519,129.704,697.490,789.882,584.695,688.340,908.238,1002.63,716.735,721.320,67.6546,66.8752\n12/29/2020,14:52,251.783,251.057,606.363,629.013,119.244,146.408,120.125,129.180,696.248,789.385,586.952,691.122,908.899,1003.46,717.180,722.196,67.3477,66.3975\n12/29/2020,14:53,250.814,250.322,608.823,632.356,119.262,146.672,120.345,129.403,699.693,794.014,593.385,699.194,913.994,1010.46,721.170,726.56,66.9330,65.9686\n12/29/2020,14:54,250.297,250.166,606.122,629.852,118.464,145.975,119.736,128.649,696.618,791.250,594.593,700.688,913.660,1010.13,723.253,727.940,67.0595,66.146\n12/29/2020,14:55,246.355,246.142,597.271,620.250,116.734,144.077,118.434,127.113,686.522,779.239,589.661,694.180,902.109,996.904,714.929,719.242,66.5781,65.4437\n12/29/2020,14:56,244.411,244.206,596.506,619.649,116.311,143.738,118.255,126.906,686.268,779.284,593.604,698.761,906.413,1002.03,717.842,722.011,65.8203,64.6744\n12/29/2020,14:57,240.505,239.879,583.014,605.377,114.048,141.038,116.229,124.458,670.380,760.030,583.919,685.853,885.174,976.707,708.776,712.584,65.1291,64.0057\n12/29/2020,14:58,239.855,239.513,579.614,601.256,113.311,140.363,115.760,124.052,666.324,756.105,584.119,686.113,882.295,973.965,713.877,717.452,64.6911,63.6007\n12/29/2020,14:59,239.100,239.015,581.417,603.274,113.606,141.232,116.004,124.612,669.045,760.398,590.743,694.412,892.014,985.824,715.425,719.083,64.7587,63.7077\n12/29/2020,15:00,234.725,234.449,573.605,594.981,112.376,139.694,114.886,123.392,659.572,748.771,586.218,688.526,881.912,974.231,704.36,708.057,64.5889,63.6541\n12/29/2020,15:01,223.801,221.959,541.396,560.428,107.294,133.072,110.566,118.290,621.424,702.177,554.838,649.181,825.25,908.242,661.792,664.875,64.2171,63.3051\n12/29/2020,15:02,225.197,225.039,537.694,559.094,106.822,133.444,110.057,118.268,619.217,703.009,557.760,655.218,832.216,921.634,675.992,679.585,64.3002,63.4196\n12/29/2020,15:03,225.350,225.834,549.684,568.732,108.406,135.410,111.399,120.030,632.667,717.046,572.302,671.929,852.470,942.572,685.343,688.115,64.7366,63.8757\n12/29/2020,15:04,222.703,223.273,548.893,569.098,108.119,135.396,111.274,119.968,632.640,718.672,576.037,677.298,856.324,949.186,681.470,683.750,65.0843,64.2512\n12/29/2020,15:05,221.312,221.869,542.794,563.332,107.060,134.313,110.207,118.749,626.011,711.492,574.266,675.403,854.169,947.640,678.643,680.714,65.3578,64.4680\n12/29/2020,15:06,214.672,214.018,524.406,543.213,104.242,130.708,107.528,115.413,604.276,685.351,557.843,654.559,826.209,914.841,652.183,653.942,65.5498,64.5950\n12/29/2020,15:07,210.069,209.247,513.551,532.320,103.094,129.394,106.145,113.764,592.699,672.748,551.472,647.081,816.124,905.234,638.715,640.627,66.1231,65.1456\n12/29/2020,15:08,210.090,210.055,512.981,532.430,103.553,130.163,106.126,113.738,592.602,674.001,555.379,652.346,818.366,909.409,640.911,642.975,67.0734,65.9773\n12/29/2020,15:09,215.135,216.33,529.528,550.148,106.473,134.280,108.186,116.478,611.898,698.224,577.637,680.014,846.393,941.954,662.855,664.799,68.9785,67.8115\n12/29/2020,15:10,215.465,216.790,528.15,549.158,106.760,134.970,107.981,116.497,610.515,697.625,580.308,683.769,847.451,944.534,662.466,663.973,70.6161,69.3901\n12/29/2020,15:11,215.219,217.003,524.737,544.754,106.927,135.301,107.597,116.100,606.980,692.601,580.221,683.026,846.643,943.667,664.366,665.499,71.3398,70.1407\n12/29/2020,15:12,213.438,215.465,519.123,538.193,107.068,135.561,107.317,115.789,600.791,685.030,577.250,678.955,842.091,938.892,661.754,662.719,72.1413,70.952\n12/29/2020,15:13,212.064,214.183,511.874,530.177,106.909,135.492,107.212,115.753,592.405,675.255,571.392,671.660,829.777,925.062,653.196,654.251,73.4553,72.2730\n12/29/2020,15:14,210.951,213.350,508.907,527.163,107.319,136.254,107.896,116.598,588.653,671.534,570.365,670.799,825.911,921.693,646.720,647.453,75.5956,74.3532\n12/29/2020,15:15,211.536,214.446,509.146,528.067,107.791,137.256,108.833,117.925,587.804,671.681,572.922,674.514,826.543,923.856,644.636,645.236,78.2358,76.7630\n12/29/2020,15:16,211.429,214.608,506.707,526.056,107.407,137.098,109.666,119.026,583.768,667.710,571.736,673.664,821.537,919.16,641.031,641.979,80.3621,78.9977\n12/29/2020,15:17,213.058,216.797,506.614,526.649,106.843,136.730,110.549,120.153,581.908,666.504,572.774,675.573,819.135,917.589,645.001,645.941,82.3068,81.0613\n12/29/2020,15:18,208.249,211.263,492.146,511.969,103.464,132.547,109.532,118.816,563.459,645.077,556.470,655.584,793.033,887.954,617.818,618.549,83.8407,82.6104\n12/29/2020,15:19,208.052,211.695,494.907,513.838,102.520,131.67,110.141,119.665,565.688,647.460,561.328,660.964,800.347,896.692,619.812,620.569,85.2809,84.0241\n12/29/2020,15:20,190.341,190.979,438.889,455.25,92.9400,118.913,103.590,111.516,498.098,565.569,493.307,576.693,699.097,779.119,524.826,525.357,86.4417,85.1681\n12/29/2020,15:21,196.396,199.518,456.997,475.510,94.5544,121.901,105.736,114.534,519.171,594.013,518.072,608.846,734.245,823.464,555.445,557.326,88.7185,87.3019\n12/29/2020,15:22,179.801,179.753,411.917,424.694,86.1163,110.421,99.4245,106.735,464.995,525.971,460.404,535.475,654.159,727.267,475.612,477.387,87.5982,86.1680\n12/29/2020,15:23,145.333,139.280,316.594,324.442,69.9108,88.8646,86.0796,90.4126,351.848,390.904,341.708,391.013,480.577,526.024,325.540,326.928,81.5069,80.2464\n12/29/2020,15:24,107.337,95.5601,198.578,202.922,50.4196,63.4077,69.2528,70.1753,214.777,232.767,201.148,225.558,284.079,304.478,163.981,166.508,72.1099,71.0228\n12/29/2020,15:25,78.8470,62.9332,120.962,123.427,37.3152,46.5291,57.6891,56.6849,125.828,133.848,110.432,122.887,165.402,172.794,46.7353,49.7758,64.4694,63.4584\n12/29/2020,15:26,69.6611,53.8239,109.300,111.739,34.4218,43.1982,54.0873,52.8905,113.068,121.852,100.353,112.687,145.912,153.125,32.1888,35.4619,59.4107,58.3835\n12/29/2020,15:27,66.8768,51.0887,110.708,113.500,34.1042,43.1152,52.6905,51.7207,115.588,126.442,106.132,119.907,153.967,162.907,31.2605,34.4737,57.0679,56.0583\n12/29/2020,15:28,68.6592,54.1122,117.644,119.638,35.3348,44.7571,52.0418,51.2782,124.186,136.022,119.249,133.827,180.460,191.653,58.4425,61.4676,55.1915,54.1432\n12/29/2020,15:29,65.3299,50.2661,97.0929,99.7707,33.0146,41.9132,49.3528,48.3585,99.9585,110.307,95.9361,108.520,146.011,154.695,32.4209,35.7163,54.7979,53.7686\n12/29/2020,15:30,66.6224,52.0133,97.0634,98.6566,34.9995,44.3672,50.7796,50.1083,97.9011,107.261,95.6435,107.032,143.935,150.661,27.3152,30.0366,57.4425,56.4722\n12/29/2020,15:31,70.8435,56.6220,101.519,104.762,38.1753,48.6871,53.5684,53.5522,101.699,113.193,100.103,113.243,144.257,152.944,20.4420,23.3199,62.8487,61.8221\n12/29/2020,15:32,83.5949,72.0212,152.413,159.252,48.4745,62.6499,61.1163,63.1270,163.673,186.255,168.332,192.933,251.392,274.640,78.0548,81.2358,68.8305,67.8097\n12/29/2020,15:33,95.7363,85.3219,150.363,158.492,50.9356,65.6378,60.7220,62.8516,155.126,177.126,160.232,183.978,221.215,239.195,92.9617,96.5673,75.7840,74.6927\n12/29/2020,15:34,118.573,114.995,269.498,280.780,66.9642,86.9624,71.4974,75.9749,283.846,326.715,300.350,347.915,310.065,338.198,218.571,220.261,82.0362,80.8636\n12/29/2020,15:35,130.039,129.268,275.289,284.945,70.0518,90.4631,70.3052,74.5299,296.005,339.999,322.843,373.501,391.882,433.596,285.734,286.087,84.0530,82.9919\n12/29/2020,15:36,119.919,110.653,110.492,116.629,46.5249,58.5465,51.2137,51.2617,106.614,117.703,112.927,127.144,143.421,150.555,116.732,121.629,80.7193,79.7418\n12/29/2020,15:37,80.3193,66.9804,102.208,108.455,42.5023,53.4436,47.8713,47.5001,100.816,112.049,109.378,123.747,136.657,144.426,7.59910,7.46514,73.2103,72.2110\n12/29/2020,15:38,72.4153,59.1261,96.3113,102.659,38.9605,48.6619,44.9031,44.1840,97.6053,108.857,107.271,121.586,133.997,142.520,7.89536,8.03261,65.7373,64.5584\n12/29/2020,15:39,65.9395,52.5959,90.7025,96.6164,35.3481,43.8983,42.2791,41.2940,93.3086,104.059,102.317,115.964,128.954,137.461,7.70280,7.75377,59.4927,58.2239\n12/29/2020,15:40,60.1866,46.5251,83.4241,88.3958,31.9518,39.5063,39.9195,38.6005,86.0243,95.4661,93.1483,105.230,118.154,125.312,7.03127,6.85854,53.7501,52.6401\n12/29/2020,15:41,56.0911,42.1476,76.6443,80.5423,29.4994,36.3378,38.0687,36.4640,78.6206,86.6164,83.9480,94.3254,106.712,112.205,5.59440,4.66694,49.7897,48.7685\n12/29/2020,15:42,53.4136,39.2892,71.1157,74.3205,27.9159,34.3024,36.7517,34.9961,72.5157,79.5228,75.9183,84.9213,97.3277,101.42,3.60944,2.16224,47.4003,46.3819\n12/29/2020,15:43,51.4171,37.0943,67.1722,69.8763,26.7736,32.9021,35.7291,33.9278,68.0995,74.4702,69.3998,77.3871,89.5735,92.5870,2.86878,1.28169,45.3377,44.3640\n12/29/2020,15:44,49.7111,35.2569,63.7865,66.2667,25.8032,31.7581,34.8461,33.0530,64.3742,70.3392,64.2914,71.6095,83.2559,85.5046,2.66634,1.04687,43.4146,42.5652\n12/29/2020,15:45,48.2699,33.7738,60.7221,62.9576,24.9607,30.8054,34.0130,32.2862,60.9035,66.4690,60.2685,66.9802,77.9422,79.3964,2.2417,0.391355,41.9056,41.1806\n12/29/2020,15:46,47.1455,32.7320,57.5816,59.6109,24.2815,30.0115,33.1400,31.4998,57.1211,62.2754,57.1503,63.5253,73.1475,74.1574,1.98001,-0.210354,40.7825,40.1315\n12/29/2020,15:47,46.4742,32.1405,56.1445,58.2046,23.8316,29.5820,32.5962,31.0115,55.4377,60.4672,56.4266,62.7974,71.5866,72.6169,2.13802,-0.136975,40.0485,39.4587\n12/29/2020,15:48,46.1052,32.1166,60.2319,63.1621,24.1007,30.1270,32.7309,31.4244,61.2829,67.8634,64.4990,72.5218,79.7023,82.7190,6.52270,4.43702,39.2162,38.6713\n12/29/2020,15:49,57.4914,48.7769,144.409,153.679,35.1101,45.6020,41.6508,42.6181,168.106,196.577,196.270,227.902,246.674,279.139,125.314,123.434,39.3782,38.7845\n12/29/2020,15:50,80.0701,78.7469,208.523,217.597,44.4084,57.5911,47.8671,50.1583,248.137,288.829,297.307,343.704,391.177,442.368,303.989,304.305,39.3972,38.7468\n12/29/2020,15:51,93.2,96.8723,265.645,276.433,52.1584,68.1690,53.5743,57.1774,321.205,376.803,390.712,455.264,517.520,589.450,425.866,426.389,39.6028,38.9355\n12/29/2020,15:52,96.7217,101.958,290.190,300.383,55.0476,71.9801,55.6398,59.7039,353.310,413.689,432.283,503.690,574.082,653.886,473.678,473.259,39.1553,38.5175\n12/29/2020,15:53,94.6848,99.4813,289.575,298.919,54.3774,71.1396,55.0463,59.0259,353.561,412.456,434.504,504.921,574.817,653.534,474.265,472.716,38.3671,37.7797\n12/29/2020,15:54,91.9772,96.1994,285.230,294.823,53.2661,69.9164,54.0887,58.0136,349.317,407.405,431.920,501.902,568.823,647.621,469.464,467.074,37.5413,36.9982\n12/29/2020,15:55,88.0832,90.8346,266.887,272.853,50.4654,66.0444,51.5896,54.9467,326.479,376.580,406.215,466.439,536.751,606.362,447.667,444.964,36.7137,36.2162\n12/29/2020,15:56,83.9887,85.4665,256.990,262.143,48.6177,63.7929,50.0385,53.2465,314.890,362.524,394.288,451.448,520.052,586.947,429.250,426.552,35.9293,35.4653\n12/29/2020,15:57,81.6599,82.5753,254.087,260.136,47.6914,62.9593,49.1955,52.5945,312.183,360.991,393.735,452.322,517.821,586.847,424.046,421.739,35.1839,34.7756\n12/29/2020,15:58,81.3180,82.5839,257.761,265.976,47.8196,63.5262,48.8864,52.6439,317.557,370.107,404.204,467.876,530.192,605.003,437.983,436.056,34.5423,34.1736\n12/29/2020,15:59,79.5470,80.3203,252.063,259.987,46.6507,62.1693,47.6542,51.4511,310.724,362.370,398.505,461.053,522.470,596.340,434.380,432.618,33.9188,33.5874\n12/29/2020,16:00,77.4354,77.6783,247.557,255.680,45.5127,60.9314,46.6165,50.4090,305.503,356.928,394.801,457.307,516.554,590.395,429.803,428.265,33.3465,33.0169\n12/29/2020,16:01,75.3910,75.2313,241.855,249.663,44.2291,59.4914,45.4341,49.1834,298.878,349.298,389.043,450.452,508.789,582.014,424.846,423.808,32.7528,32.4391\n12/29/2020,16:02,73.0733,72.2679,236.419,244.067,42.9102,57.9936,44.3116,48.0070,292.560,342.218,383.546,444.056,500.364,572.860,417.799,417.066,32.0705,31.7720\n12/29/2020,16:03,70.8988,69.6798,230.924,238.233,41.8780,56.5211,43.1043,46.7782,286.002,334.693,377.820,437.178,491.971,563.348,412.467,411.871,31.3520,31.0644\n12/29/2020,16:04,68.5717,66.8815,225.252,232.128,44.0792,55.8609,41.8522,45.5429,279.176,326.699,371.420,429.266,482.533,552.132,406.057,405.575,30.5797,30.3169\n12/29/2020,16:05,65.7050,63.2878,216.199,221.767,49.1873,58.1636,40.3207,43.8522,267.943,312.307,358.543,412.513,465.306,530.665,392.396,392.147,29.8096,29.5563\n12/29/2020,16:06,62.5922,59.3487,205.78,209.895,47.6657,63.4584,38.6745,42.0566,254.929,295.873,342.989,392.424,444.552,504.917,373.208,373.172,29.0675,28.8548\n12/29/2020,16:07,60.5428,56.9017,199.212,202.731,39.4393,59.9182,37.2778,40.7067,246.699,286.333,334.498,381.873,433.971,492.473,365.801,366.005,28.4134,28.2293\n12/29/2020,16:08,58.4198,54.2835,191.647,194.348,35.8224,51.2422,35.7763,39.2519,237.057,274.663,323.638,368.009,419.360,474.426,355.135,355.433,27.8555,27.6633\n12/29/2020,16:09,56.1856,51.5093,185.768,188.063,34.8521,48.7954,34.5341,38.0264,229.549,265.991,315.662,358.221,407.185,459.725,342.939,343.315,27.3249,27.1625\n12/29/2020,16:10,54.7655,49.9811,181.985,184.442,33.9832,48.2758,33.4666,37.0369,225.092,261.485,312.220,354.673,402.924,456.003,340.347,341.128,26.8322,26.6742\n12/29/2020,16:11,53.0869,48.0055,176.410,178.541,34.3803,46.3956,32.1645,35.8409,218.083,253.463,304.952,345.948,393.417,445.035,332.792,333.746,26.3274,26.2107\n12/29/2020,16:12,51.4877,46.1169,171.329,173.129,31.4646,45.0458,30.9522,34.6776,211.885,246.359,298.598,338.356,385.229,435.577,326.624,327.625,25.8255,25.6958\n12/29/2020,16:13,49.7356,44.0871,164.786,165.696,27.5432,41.2888,29.6352,33.2882,203.431,235.673,288.913,325.710,372.163,419.075,317.305,318.281,25.3264,25.2158\n12/29/2020,16:14,47.6369,41.5831,156.032,155.497,25.3332,37.7844,28.1136,31.6399,191.990,220.925,274.678,306.626,353.28,394.418,302.053,303.116,24.7743,24.6883\n12/29/2020,16:15,45.2797,38.6888,142.599,140.432,23.0131,34.8286,26.1929,29.4280,172.744,196.655,248.667,273.650,313.474,344.255,273.454,274.743,24.2117,24.1661\n12/29/2020,16:16,40.7556,33.3956,103.049,101.140,18.1740,28.8410,22.7005,25.4956,118.585,133.224,169.113,182.080,202.906,216.940,202.598,203.672,23.6068,23.5892\n12/29/2020,16:17,33.9370,25.5593,56.3143,57.4501,13.2203,22.8173,19.0884,21.6057,58.0354,66.8956,81.0019,87.3391,94.4952,99.0097,101.672,101.630,22.9951,22.9604\n12/29/2020,16:18,29.1733,20.3051,32.9339,36.8924,10.4545,19.6052,17.1028,19.6264,29.3529,38.0523,38.1036,45.7953,46.2761,50.3407,17.1979,17.3077,22.401,22.3592\n12/29/2020,16:19,28.0289,19.4163,26.8722,31.8760,9.39146,18.5766,16.0950,18.8302,22.3538,31.8273,27.4602,36.6848,35.0279,39.8134,2.37995,3.77168,21.8704,21.8250\n12/29/2020,16:20,27.4716,19.1312,25.6759,30.7618,8.90623,18.1760,15.3267,18.2993,20.6858,30.5928,25.6774,35.4166,33.5782,38.4784,1.56524,3.24336,21.4051,21.3475\n12/29/2020,16:21,26.9286,18.8519,24.8179,29.8480,8.43428,17.8007,14.5984,17.7718,19.4441,29.5462,24.5121,34.4883,32.3470,37.3152,0.967781,2.64164,20.8988,20.8452\n12/29/2020,16:22,26.2384,18.4676,23.8838,28.8840,7.90936,17.3568,13.8400,17.2704,18.3113,28.5308,23.3201,33.4918,31.0987,36.1311,0.498703,2.03993,20.3155,20.2760\n12/29/2020,16:23,25.4763,18.0052,22.8819,27.8115,7.34031,16.8768,13.1915,16.7166,17.2150,27.3936,22.0854,32.3692,29.7047,34.8002,0.172818,1.41866,19.6386,19.6240\n12/29/2020,16:24,24.7800,17.4828,21.8759,26.6763,6.81097,16.3968,12.5678,16.1857,16.1549,26.1938,20.9785,31.156,28.3018,33.3808,-0.207382,0.895225,18.9452,18.9280\n12/29/2020,16:25,24.0569,16.9905,20.8867,25.5746,6.29045,15.9096,11.8993,15.6483,15.0378,25.0532,19.8716,29.9946,26.9547,32.0159,-0.474016,0.435383,18.2753,18.2437\n12/29/2020,16:26,23.3353,16.4860,19.8891,24.5187,5.68170,15.4079,11.1409,15.042,13.8479,23.9473,18.5465,28.7619,25.5948,30.6009,-0.632019,0.014676,17.6096,17.5801\n12/29/2020,16:27,22.5430,15.9186,18.8704,23.4462,5.04648,14.8413,10.3227,14.3964,12.7151,22.8066,17.1469,27.3578,24.1320,29.064,-0.775212,-0.386463,16.9061,16.8953\n12/29/2020,16:28,21.7976,15.2881,17.7628,22.2275,4.53037,14.1952,9.56434,13.6918,11.5927,21.5373,15.6408,25.7078,22.5233,27.2416,-0.844340,-0.812062,16.1452,16.1524\n12/29/2020,16:29,20.8889,14.5015,16.5285,20.8002,3.86427,13.4012,8.83591,12.9414,10.4028,20.0210,14.007,23.6857,20.5585,25.0118,-0.888780,-1.09090,15.2232,15.2780\n12/29/2020,16:30,19.7493,13.3546,15.0363,18.9932,2.99967,12.3437,8.03766,12.0664,8.92709,18.1188,12.0114,21.1493,18.2247,22.2235,-0.913469,-1.23277,14.0539,14.1504\n12/29/2020,16:31,18.4486,11.9615,13.3455,16.965,1.98508,11.0877,7.13464,11.1063,7.24875,16.0427,9.79752,18.4026,15.7321,19.1915,-0.962847,-1.47248,12.6736,12.8437\n12/29/2020,16:32,17.1596,10.6405,11.7561,15.0578,0.966079,9.84618,6.16177,10.0839,5.80943,14.1648,7.93492,16.1250,13.6343,16.6509,-0.982598,-1.64859,11.3520,11.5420\n12/29/2020,16:33,16.0966,9.57165,10.4710,13.5429,0.189687,8.81754,5.27368,9.05812,4.77538,12.6972,6.73219,14.5591,12.1457,14.9376,-0.967789,-1.81492,10.2554,10.4851\n12/29/2020,16:34,15.2998,8.83006,9.52838,12.4829,-0.449958,8.05241,4.33073,8.08156,3.93360,11.6505,5.95522,13.5983,11.2276,13.9172,-1.00235,-1.99104,9.42169,9.65205\n12/29/2020,16:35,14.7246,8.29563,8.77172,11.6775,-0.913153,7.50021,3.43267,7.27212,3.13858,10.8750,5.41773,12.9707,10.5798,13.2747,-0.972727,-2.13779,8.81372,9.00119\n12/29/2020,16:36,14.1837,7.90532,8.11651,11.0473,-1.33665,7.09599,2.67429,6.70519,2.36953,10.2386,4.81637,12.5016,9.99634,12.7456,-0.888786,-2.42153,8.34540,8.50791\n12/29/2020,16:37,13.6554,7.57507,7.49934,10.4964,-1.71603,6.74951,2.00074,6.24967,1.68362,9.71693,4.27886,12.0519,9.48586,12.3005,-0.834473,-2.47534,7.89771,8.08094\n12/29/2020,16:38,13.1670,7.30187,6.98361,9.99562,-2.08659,6.41385,1.44193,5.88919,1.10163,9.23006,3.73603,11.6119,8.92817,11.8512,-0.770284,-2.57319,7.49621,7.63740\n12/29/2020,16:39,12.6973,7.01963,6.50171,9.51986,-2.42628,6.12151,0.997878,5.53198,0.659940,8.79188,3.20916,11.1719,8.43914,11.4230,-0.696218,-2.69060,7.14926,7.27155\n12/29/2020,16:40,12.2115,6.68336,6.07475,9.06496,-2.75273,5.83277,0.623675,5.20754,0.218248,8.35022,2.67164,10.7028,7.87287,10.9527,-0.686342,-2.78843,6.75920,6.86952\n12/29/2020,16:41,11.6453,6.27503,5.55902,8.51823,-2.98212,5.48627,0.224523,4.86999,-0.233838,7.88768,2.05962,10.0266,7.13922,10.2724,-0.676466,-2.99879,6.29187,6.40555\n12/29/2020,16:42,10.8690,5.74960,4.92491,7.89636,-3.17181,5.07119,-0.014968,4.51277,-0.758673,7.33470,1.35179,9.16598,6.29404,9.39887,-0.720905,-3.22382,5.69754,5.79525')

df = pd.read_csv(TESTDATA, sep=",")

In [8]:
df['measurement_ts'] = pd.to_datetime(df['DATE (MM/DD/YYYY)'] + ' ' + df['MST']).dt.tz_localize('MST')
df.drop(['DATE (MM/DD/YYYY)', 'MST'], axis=1, inplace=True)
df['station_id'] = 1

In [9]:
df.columns = df.columns.str.lower()\
                       .str.replace(r"\(.*\)|\[.*\]", '')\
                       .str.replace('li-200', 'li200')\
                       .str.strip().str.replace('-|\W+', '_')

In [10]:
engine = create_engine(f'postgresql://{username}:{password}@{hostname}:5432/{database}')

In [11]:
df.head()

,global_psp,global_li200,global_40_south_cmp22,global_40_south_li200r,global_90_north_psp,global_90_north_li200,global_90_east_psp,global_90_east_li200,global_90_south_psp,global_90_south_li200,global_90_west_psp,global_90_west_li200,global_normal_cmp22,global_normal_li200r,direct_snip,direct_nip,diffuse_cm22_2,diffuse_cm22_1,measurement_ts,station_id
0,9.60577,2.89732,5.72671,7.09437,1.50504,5.16036,5.86544,9.50543,4.43471,7.68627,1.49090,4.56097,8.41596,9.76368,0.553023,-0.210355,5.04128,5.00532,2020-12-29 07:24:00-07:00,1
1,10.13790,3.17354,6.42456,7.74575,2.25976,5.57925,6.86880,10.43990,5.31332,8.43088,2.18842,4.97855,9.63116,10.80610,0.592524,0.704445,5.39809,5.53059,2020-12-29 07:25:00-07:00,1
2,10.87430,3.52482,7.25354,8.44308,3.54853,6.00175,8.57102,11.58750,6.71704,9.30772,3.39178,5.37993,11.34020,12.08800,1.575130,2.480230,5.67426,5.98470,2020-12-29 07:26:00-07:00,1
3,11.76940,3.83106,8.09946,9.14878,4.97414,6.40983,10.58780,12.76790,8.55749,10.20550,4.73893,5.78458,13.22950,13.43720,2.142960,3.947830,5.93630,6.36775,2020-12-29 07:27:00-07:00,1
4,12.57100,4.03222,8.91154,9.84614,6.28499,6.78902,12.46470,13.83690,10.22120,11.05450,5.86776,6.18599,14.93840,14.66030,2.513290,4.916440,6.22500,6.73611,2020-12-29 07:28:00-07:00,1


In [12]:
df.to_sql('irradiance', engine, if_exists='append', index=False)